In [1]:
# %matplotlib widget

# %matplotlib inline
%matplotlib qt5

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

from qick import *
from qick.helpers import gauss

import time
import os
import sys
sys.path.append('/home/xilinx/jupyter_notebooks/')
import scipy as sp
import json
from scipy.fft import fft, fftfreq

from slab.instruments import *
from slab.experiment import Experiment
from slab.datamanagement import SlabFile
from slab import get_next_filename, AttrDict

# Figure params
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [10,6]
plt.rcParams.update({'font.size': 14})
mpl.rcParams['figure.dpi'] = 150

import Pyro4.util

default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
linestyle_cycle=['solid', 'dashed', 'dotted', 'dashdot']
marker_cycle = ['o', '*', 's', '^']

Could not import QickInstrument (will only work if running on rfsoc). If running on rfsoc, try install/update qick package
Warning failed to import AWG81180A
Warning failed to import Tek5014
Warning failed to import Tek70001
Warning failed to import M8195A


# Load experiments

In [3]:
# %aimport experiments
# NOTE: ADDING NEW METHODS WILL NOT BE UPDATED
import experiments as meas

imported experiments.clifford_averager_program
imported experiments.fitting
imported experiments.four_qubit.fourQ_state_tomo
imported experiments.qram_protocol_timestepped
imported experiments.single_qubit.acstarkshift_spectroscopy
imported experiments.single_qubit.amplitude_rabi
imported experiments.single_qubit.length_rabi
imported experiments.single_qubit.pulse_probe_spectroscopy
imported experiments.single_qubit.resonator_spectroscopy
imported experiments.single_qubit.rfsoc_tof_calibration
imported experiments.single_qubit.single_shot
imported experiments.single_qubit.t1
imported experiments.single_qubit.t2_ramsey
imported experiments.three_qubit.threeQ_state_tomo
imported experiments.two_qubit.amplitude_rabi_EgGf
imported experiments.two_qubit.amplitude_rabi_f0g1
imported experiments.two_qubit.amprabi_opt_ctrl_state_prep
imported experiments.two_qubit.crosstalk_echo_calib
imported experiments.two_qubit.length_rabi_EgGf
imported experiments.two_qubit.length_rabi_F0G1
imported exper

### Set save data

In [4]:
save_data = False

In [5]:
def prev_data(expt_path, filename=None):
    """Reopen saved data"""
    temp_data_file = expt_path
    if filename is not None: temp_data_file = os.path.join(expt_path, filename)
    print(temp_data_file)
    with SlabFile(temp_data_file) as a:
        attrs = dict()
        for key in list(a.attrs):
            attrs.update({key:json.loads(a.attrs[key])})
        keys = list(a)
        temp_data = dict()
        for key in keys:
            temp_data.update({key:np.array(a[key])})
        print(f'Extracted data and attrs {list(a.attrs)}')
    return temp_data, attrs

In [6]:
# expt_path=os.path.join(os.getcwd(), 'data', 'data_240617')
data_path = 'S:\\QRAM\\qram_4QR2\\'
expt_path = os.path.join(data_path, 'data', 'data_250119')
print('Data will be stored in', expt_path)

Data will be stored in S:\QRAM\qram_4QR2\data\data_250119


In [7]:
# config_file = 'config_zcu216.yml'
# config_file = 'config_q3diamond.yml'
# config_file = 'config_q3diamond_full688_reset.yml'
config_file = 'config_q3diamond_full688and638_reset.yml'
config_path = os.path.join('s:\\Connie\\experiments\\qram_tprocv1_expts\\configs\\', config_file)
print('Config will be', config_path)

Config will be s:\Connie\experiments\qram_tprocv1_expts\configs\config_q3diamond_full688and638_reset.yml


# Load config

In [8]:
import yaml
with open(config_path, 'r') as cfg_file:
    yaml_cfg = yaml.safe_load(cfg_file)
yaml_cfg = AttrDict(yaml_cfg)

# Setup experiment hardware

Set in config:
- LO alias, frequency, power
- SOC dac channels, nyquist zones

In [9]:
# im = InstrumentManager(ns_address='192.168.14.1') # Chicago lab
# im = InstrumentManager(ns_address='192.168.137.1') # SLAC lab
# im = InstrumentManager(ns_address='10.108.30.56') # Spilker Fermium
im = InstrumentManager(ns_address='10.108.30.75') # Spilker Vibranium

print(list(im))
# print(im)

['Pyro.NameServer', 'Qick_QRAM']


In [10]:
soc = QickConfig(im[yaml_cfg['aliases']['soc']].get_cfg())
print(soc)
# print(soc.description())
# print(soc.dump_cfg())

# soc = QickConfig(im['Qick187'].get_cfg())

QICK running on ZCU216, software version 0.2.272

Firmware configuration (built Sat Oct  5 10:21:47 2024):

	Global clocks (MHz): tProcessor 430.080, RF reference 245.760

	7 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 0 is 0_230, on JHC3
	1:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 1 is 1_230, on JHC4
	2:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 2 is 2_230, on JHC3
	3:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 3 is 3_230, on JHC4
	4:	axis_signal_gen_v6 - envelope memory 32768 samples (4.762 us)
		fs=6881.280 MHz, fabric=430.080 MH

In [11]:
# lo1 = im[yaml_cfg.aliases.readout_LO]
# lo1.open_device()
# print(lo1.get_id())

# # TURN OFF
# lo1.set_standby(True)
# lo1.set_output_state(False)

# # # TURN ON
# # lo1.set_standby(False)
# # lo1.set_output_state(True)
# # lo_freq = float(yaml_cfg.hw.lo.readout.frequency)
# # lo_power = float(yaml_cfg.hw.lo.readout.power)
# # lo1.set_frequency(lo_freq) # Set LO frequency
# # print(f'set freq to [MHz] {lo_freq*1e-6}')
# # lo1.set_power(lo_power) # Set LO power
# # print('set power to', lo_power)

## Set TWPA parameters

Set in config:
- LO alias, frequency, power
- SOC dac channels, nyquist zones

In [14]:
"""
TWPA ON
"""

print('Using config file', config_file)
with open(config_path, 'r') as cfg_file:
    yaml_cfg = yaml.safe_load(cfg_file)
yaml_cfg = AttrDict(yaml_cfg)
im = InstrumentManager(ns_address='10.108.30.56') # Spilker Fermium

lo_twpa = im[yaml_cfg.aliases.twpa_LO]
# lo_twpa = SignalCore(name="SignalCore", address='10003494')
lo_twpa.open_device()
print(lo_twpa.get_id())

# TURN ON
lo_twpa.set_standby(False)
lo_twpa.set_output_state(True)
twpa_freq = float(yaml_cfg.hw.lo.twpa.frequency)
twpa_power = float(yaml_cfg.hw.lo.twpa.power)
lo_twpa.set_frequency(twpa_freq) # Set LO frequency
print(f'set twpa pump freq to {twpa_freq*1e-6} MHz')
lo_twpa.set_power(twpa_power) # Set LO power
print(f'set twpa pump power to {twpa_power} dBm')

Using config file config_q3diamond_full688and638_reset.yml
Default Instrument SC_26C1
set twpa pump freq to 8194.0 MHz
set twpa pump power to -0.6 dBm


In [13]:
"""
TWPA OFF
"""
im = InstrumentManager(ns_address='10.108.30.56') # Spilker Fermium

lo_twpa = im[yaml_cfg.aliases.twpa_LO]
# lo_twpa = SignalCore(name="SignalCore", address='10003494')
lo_twpa.open_device()
print(lo_twpa.get_id())

# TURN OFF
lo_twpa.set_standby(True)
lo_twpa.set_output_state(False)

Default Instrument SC_26C1


0

# Qubit/resonator to probe

In [19]:
all_results = [dict(), dict(), dict(), dict()]
saved_files = []

In [20]:
qubit_i = 2
results_Qi = all_results[qubit_i]

In [21]:
full_mux_expt = False
resonator_reset = [0, 2, 3] if full_mux_expt else None
use_robust_pulses = True

## Set reps and rounds

In [22]:
reps = 100
rounds = 10

# RFSoC Calibration

In [ ]:
tof = meas.ToFCalibrationExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"adc_trig_offset_calibration_qubit{qubit_i}",
    config_file=config_path,
)

tof.cfg.expt = dict(
    pulse_length=0.5, # [us]
    readout_length=1.0, # [us]
    trig_offset=0, # [clock ticks]
    # gain=32000,
    gain=0.8, # [MUX units: -1 to 1]
    frequency=tof.cfg.device.readout.frequency[qubit_i]-15, # [MHz]
    reps=reps*rounds, # Number of averages per point
    # qubit=qubit_i,
    qubit=qubit_i,
)
tof.cfg.device.readout.relax_delay=0.1 # wait time between experiments [us]
# print(tof.cfg)

try:
    tof.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

In [ ]:
tof.display(adc_trig_offset=150)

# Initial single tone resonator spectroscopy

In [86]:
rspec = meas.ResonatorSpectroscopyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"resonator_spectroscopy_qubit{qubit_i}",
    config_file=config_path,
)
rspec.cfg = AttrDict(deepcopy(yaml_cfg))

span = 1.0 # MHz
npts = 150
rspec.cfg.expt = dict(
    start=rspec.cfg.device.readout.frequency[qubit_i]-span/2, # [MHz]
    step=span/npts,
    expts=npts,
    reps=reps*rounds,
    pulse_e=False, # add ge pi pulse prior to measurement
    pulse_f=False, # add ef pi pulse prior to measurement
    qTest=qubit_i,
    qZZ=None,

    resonator_reset=resonator_reset,
    full_mux_expt=full_mux_expt,
)
# rspec.cfg.device.readout.gain[qubit_i] /= 3
# rspec.cfg.device.readout.readout_length = [3*rspec.cfg.device.readout.readout_length[qubit_i]]*4

rspec.cfg.device.readout.relax_delay = 25 # Wait time between experiments [us] - should be 5x resonator lifetime for good shape

try:
    rspec.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

  0%|          | 0/150 [00:00<?, ?it/s]

In [87]:
# TESTING
# # rspec = meas.ResonatorSpectroscopyExperiment(config_file=config_path,) 
# temp_data, attrs = prev_data(expt_path, '00001_resonator_spectroscopy_qubit2.h5')
# rspec.data = temp_data

rspec.data = rspec.analyze(fit=True)
rspec_fit = rspec.data['fit']
# if not isinstance(rspec_fit, (list, np.ndarray)):
#     rspec.display(fit=False)
#     assert False, 'Fit failed, please manually fix!'
rspec.display(fit=True)
# yaml_cfg.device.readout.frequency[qubit_i] = rspec_fit[0]

res_freq_g = rspec.data['xpts'][np.argmin(rspec.data['amps'])]
# res_freq_g = rspec.data['xpts'][np.argmax(rspec.data['amps'])]
yaml_cfg.device.readout.frequency[qubit_i] = res_freq_g

print(f'*Set cfg res {qubit_i} freq to {yaml_cfg.device.readout.frequency[qubit_i]}*')
results_Qi.update({'Res freq':float(yaml_cfg.hw.lo.readout.frequency)*1e-6 + yaml_cfg.device.readout.lo_sideband[qubit_i]*(yaml_cfg.device.readout.frequency[qubit_i] + yaml_cfg.hw.soc.dacs.readout.mixer_freq[qubit_i])})
results_Qi.update({'Res freq (yaml)':yaml_cfg.device.readout.frequency[qubit_i]})
results_Qi.update({'Res kappa':abs(rspec_fit[0]*(1/rspec_fit[1] + 1/rspec_fit[2]))})
print(f'qubit {qubit_i}', results_Qi)

electrical delay phase correction (no mux support) (deg) 4.729156607233271
param guess [7702.3649000000005, 38511.8245, 38511.8245, 0, 0.3000527206662833, 0.7707919684037302]
param bounds ([7701.874900000001, 0, 0, -6.283185307179586, -0.45007908099942495, -inf], [7702.8549, 1000000000.0, 100000000.0, 6.283185307179586, 0.45007908099942495, 1.052148575800202])
popt [ 7.70240024e+03  6.74723299e+04  5.48738880e+04 -4.51068724e-01
  2.52927528e-02 -8.35446851e-02]

Freq with minimum transmission: 502.35490000000067
Freq with maximum transmission: 502.848233333334
From fit:
	f0: 7702.400243615755
	Qi: 67472.32986604629 	 kappa_i/2pi: 0.11415642914521304
	Qe: 54873.887972644814 	 kappa_e/2pi: 0.1403654912780279
	Q0: 30262.22743725783
	kappa [MHz]: 0.25452192042324095
	phi [radians]: -0.4510687241471329
baseline 63.29819704922005
*Set cfg res 2 freq to 502.36156666666733*
qubit 2 {'Qubit EF freq': 4574.2134872558845, 'T2R EF': 15.233515280933435, 'T2R EF err': 1.6586265411281977, 'Pi EF pul

In [126]:
if save_data: saved_files.append(rspec.save_data())

# Qubit ge spectroscopy

## Pulse Probe (qubit) spectroscopy

In [231]:
qspec = meas.PulseProbeSpectroscopyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"qubit_spectroscopy_qubit{qubit_i}",
    config_file=config_path,
)
qspec.cfg = AttrDict(deepcopy(yaml_cfg))

span = 50
npts = 250

qspec.cfg.expt = dict(
    start=np.reshape(yaml_cfg.device.qubit.f_ge, (4,4))[qubit_i, qubit_i]-span/2, # [MHz]
    step=span/npts,
    expts=npts,
    reps=reps, # Number of averages per point
    rounds=rounds, # Number of start to finish sweeps to average over
    length=1, # Qubit probe constant pulse length [us]
    gain=int(np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))[qubit_i, qubit_i]/10), # Qubit pulse gain
    pulse_type='gauss',
    qTest=qubit_i,
    qZZ=None,
    checkEF=False,
)
qspec.cfg.device.readout.relax_delay = 10 # Wait time between experiments [us]
qspec.go(analyze=False, display=False, progress=True, save=False)

Running pulse probe on Q3 


  0%|          | 0/10 [00:00<?, ?it/s]

In [232]:
# # TESTING
# # qspec = meas.PulseProbeSpectroscopyExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00001_qubit_spectroscopy_qubit0.h5')
# qspec.data = temp_data

best_signs = [1, 0, 0]
best_fit_err_i = np.inf
best_fit_err_q = np.inf
for sign in [1, -1]:
    try: 
        qspec.analyze(fit=True, signs=[1, sign, sign])
        fit_err_i = qspec.data['fit_err_avgi'][2][2]
        fit_err_q = qspec.data['fit_err_avgq'][2][2]
        if fit_err_i < best_fit_err_i:
            best_signs[1] = sign
            best_fit_err_i = fit_err_i
        if fit_err_q < best_fit_err_q:
            best_signs[2] = sign
            best_fit_err_q = fit_err_q
    except: continue
if best_fit_err_i == np.inf and best_fit_err_q == np.inf:
    qspec.display(fit=False)
    assert False, 'Fit failed, please manually fix!'

qspec.analyze(fit=True, signs=best_signs)
qspec.display(fit=True, signs=best_signs)

qspec_fit, qspec_fit_err = meas.fitting.get_best_fit(qspec.data)
yaml_cfg.device.qubit.f_ge = np.reshape(yaml_cfg.device.qubit.f_ge, (4,4))
yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i] = qspec_fit[2]
print(f'*Set qubit {qubit_i} f_ge to {yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i]}*')
results_Qi.update({'Qubit freq':yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
yaml_cfg.device.qubit.f_ge = yaml_cfg.device.qubit.f_ge.flatten().tolist()
print(results_Qi)

Found peak in amps at [MHz] 4377.961071249494, HWHM 0.2889794375179536
Found peak in I at [MHz] 4377.9777077297285, HWHM -0.29757487485255224
Found peak in Q at [MHz] 4377.95260123418, HWHM 0.2850932730169064
best 0 0.02797902174640183
*Set qubit 3 f_ge to 4377.961071249494*
{'Qubit freq': 4377.961071249494, 'T1': 31.07377161991536, 'T1 err': 1.7658704555417863, 'T2R': 32.06829908349514, 'T2R err': 1.628634280696754, 'Qubit EF freq': 4204.964704068256, 'T2R EF': 14.743982948593079, 'T2R EF err': 1.0729044094517184, 'Res freq': 6971.153333333334, 'Res freq (yaml)': -228.84666666666666, 'Res kappa': 0.2518690613409693}


In [134]:
if save_data: saved_files.append(qspec.save_data())

## Pulse Probe (qubit) spectroscopy round 2

In [89]:
qspec2 = meas.PulseProbeSpectroscopyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"qubit_spectroscopy_qubit{qubit_i}",
    config_file=config_path,
)
qspec2.cfg = AttrDict(deepcopy(yaml_cfg))

span = 15
npts = 250

qspec2.cfg.expt = dict(
    start=np.reshape(qspec2.cfg.device.qubit.f_ge, (4,4))[qubit_i, qubit_i]-span/2, # [MHz]
    step=span/npts,
    expts=npts,
    reps=reps, # Number of averages per point
    rounds=rounds, # Number of start to finish sweeps to average over
    length=1, # Qubit probe constant pulse length [us]
    gain=int(np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))[qubit_i, qubit_i]/80), # Qubit pulse gain
    pulse_type='gauss',
    qTest=qubit_i,
    qZZ=None,
    checkEF=False,
)
qspec2.cfg.device.readout.relax_delay = 10 # Wait time between experiments [us]
qspec2.go(analyze=False, display=False, progress=True, save=False)

Running pulse probe on Q2 


  0%|          | 0/10 [00:00<?, ?it/s]

In [90]:
# # TESTING
# # qspec2 = meas.PulseProbeSpectroscopyExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00001_qubit_spectroscopy_qubit0.h5')
# qspec2.data = temp_data

best_signs = [1, 0, 0]
best_fit_err_i = np.inf
best_fit_err_q = np.inf
for sign in [1, -1]:
    try: 
        qspec2.analyze(fit=True, signs=[1, sign, sign])
        fit_err_amps = qspec2.data['fit_err_amps'][2][2]
        fit_err_i = qspec2.data['fit_err_avgi'][2][2]
        fit_err_q = qspec2.data['fit_err_avgq'][2][2]
        if fit_err_i < best_fit_err_i:
            best_signs[1] = sign
            best_fit_err_i = fit_err_i
        if fit_err_q < best_fit_err_q:
            best_signs[2] = sign
            best_fit_err_q = fit_err_q
    except: continue
if fit_err_amps == np.inf and best_fit_err_i == np.inf and best_fit_err_q == np.inf:
    qspec2.display(fit=False)
    assert False, 'Fit failed, please manually fix!'

qspec2.analyze(fit=True, signs=best_signs)
qspec2.display(fit=True, signs=best_signs)

qspec2_fit, qspec2_fit_err = meas.fitting.get_best_fit(qspec2.data)
yaml_cfg.device.qubit.f_ge = np.reshape(yaml_cfg.device.qubit.f_ge, (4,4))
yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i] = qspec2_fit[2]
qubit_hwhm = qspec2_fit[3]
print(f'*Set qubit {qubit_i} f_ge to {yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i]}*')
results_Qi.update({'Qubit freq':yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
yaml_cfg.device.qubit.f_ge = yaml_cfg.device.qubit.f_ge.flatten().tolist()
print(results_Qi)

Found peak in amps at [MHz] 4759.4495420551475, HWHM 0.3657593779895565
Found peak in I at [MHz] 4759.455401458371, HWHM 0.37003096061515717
Found peak in Q at [MHz] 4759.446958312547, HWHM 0.3654137877181693
best 0 0.02295267445702378
*Set qubit 2 f_ge to 4759.4495420551475*
{'Res freq': 7702.361566666667, 'Res freq (yaml)': 502.36156666666733, 'Res kappa': 0.2329092543753419, 'Qubit freq': 4759.4495420551475}


In [91]:
if save_data: saved_files.append(qspec2.save_data())

## Length Rabi

In [22]:
lengthrabi = meas.LengthRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"length_rabi_qubit{qubit_i}",
    config_file=config_path,
)
lengthrabi.cfg = AttrDict(deepcopy(yaml_cfg))

lengthrabi.cfg.expt = dict(
    start=0, # pulse length (sigma) [us]
    step=soc.cycles2us(1), # [us]
    # step=0.001,
    expts=121,
    reps=reps*rounds,
    pulse_type='gauss',
    checkEF=False, 
    qTest=qubit_i,
    qZZ=None,
)
# lengthrabi.cfg.device.readout.relax_delay = 50 # Wait time between experiments [us]
# lengthrabi.pi_ge.gain[4*qubit_i+qubit_i] = 5000 # qubit gain [DAC units],

try:
    lengthrabi.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

Running length rabi on Q0 


  0%|          | 0/121 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [53]:
# # TESTING
# # lengthrabi = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_qubit0.h5')
# lengthrabi.data = temp_data

lengthrabi.analyze(fit=True)

lengthrabi_fit, lengthrabi_fit_err = meas.fitting.get_best_fit(lengthrabi.data) #, fitfunc=fitter.decaysin)

lengthrabi.display(fit=True)
# p = lengthrabi_fit
# if p[2] > 180: p[2] = p[2] - 360
# elif p[2] < -180: p[2] = p[2] + 360
# if p[2] < 0: pi_len = (1/2 - p[2]/180)/2/p[1]
# else: pi_len = (3/2 - p[2]/180)/2/p[1]
# pi_len = np.max((0.9*pi_len, 0.015)) # since we run the length rabi at a < max gain

# lengthrabi_decay = p[3]

# print(f'Will use sigma {pi_len} for amplitude rabi')

best 0 0.12566888714669036
Pi length from avgi data [us]: 0.09687820774635618
	Pi/2 length from avgi data [us]: 0.04843910387317809

Pi length from avgq data [us]: 0.09630389472058899
Pi/2 length from avgq data [us]: 0.048151947360294496


In [ ]:
# if save_data: saved_files.append(lengthrabi.save_data())

## Amplitude Rabi

In [24]:
pi_len = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.sigma, (4,4))[qubit_i, qubit_i]
# pi_len = 0.008
print('qubit', qubit_i, 'pi_len', pi_len)

qubit 0 pi_len 0.02


In [25]:
int(5*np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))[qubit_i, qubit_i])


9245

In [26]:
amprabi = meas.AmplitudeRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"amp_rabi_qubit{qubit_i}",
    config_file=config_path,
)
amprabi.cfg = AttrDict(deepcopy(yaml_cfg))

span = min((32000, int(5*np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))[qubit_i, qubit_i])))
npts = 50

amprabi.cfg.expt = dict(
    start=0, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts,
    reps=reps*rounds,
    rounds=1,
    sigma_test=pi_len, # gaussian sigma for pulse length - overrides config [us]
    # sigma_test=0.2,
    checkEF=False, 
    qTest=qubit_i,
    qZZ=None,
    # pulse_type='gauss', 
    pulse_type='robust',

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
    use_robust_pulses=use_robust_pulses,
)
amprabi.go(analyze=False, display=False, progress=True, save=False)

Running amp rabi on Q0 


  0%|          | 0/50000 [00:00<?, ?it/s]

(4, 50, 1000)


In [28]:
amprabi.cfg.expt

{'start': 0,
 'step': 184,
 'expts': 50,
 'reps': 1000,
 'rounds': 1,
 'sigma_test': 0.02,
 'checkEF': False,
 'qTest': 0,
 'qZZ': None,
 'pulse_type': 'robust',
 'full_mux_expt': False,
 'resonator_reset': None,
 'use_robust_pulses': True,
 'n_trig': 1,
 'n_init_readout': 0}

In [27]:
# # TESTING
# # amprabi = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_qubit0.h5')
# amprabi.data = temp_data

# amprabi.display(fit=False)

amprabi.analyze(fit=True)

amprabi_fit, amprabi_fit_err = meas.fitting.get_best_fit(amprabi.data)

p = amprabi_fit
if p[2] > 180: p[2] = p[2] - 360
elif p[2] < -180: p[2] = p[2] + 360
if p[2] < 0: pi_gain = (1/2 - p[2]/180)/2/p[1]
else: pi_gain = (3/2 - p[2]/180)/2/p[1]

if pi_gain > 32000:
    amprabi.display(fit=False)
    print(f'WARNING: Fit pi pulse gain is long ({pi_gain}), please double check!')
    pi_gain = 30000
else: amprabi.display(fit=True)
yaml_cfg.device.qubit.pulses.pi_ge.sigma[qubit_i] = amprabi.cfg.expt.sigma_test

print(f'*Set pi pulse sigma for qubit {qubit_i} to {np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.sigma, (4,4))[qubit_i, qubit_i]}*')
yaml_cfg.device.qubit.pulses.pi_ge.gain = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))
yaml_cfg.device.qubit.pulses.pi_ge.gain[qubit_i, qubit_i] = int(pi_gain)
print(f'*Set pi pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ge.gain[qubit_i, qubit_i]}*')
results_Qi.update({'Pi pulse gain':yaml_cfg.device.qubit.pulses.pi_ge.gain[qubit_i, qubit_i]})
print(f'*New pi_ge.gain\n', yaml_cfg.device.qubit.pulses.pi_ge.gain)
yaml_cfg.device.qubit.pulses.pi_ge.gain = yaml_cfg.device.qubit.pulses.pi_ge.gain.flatten().astype(int).tolist()
print(f'*New pi_ge.gain flattened\n{yaml_cfg.device.qubit.pulses.pi_ge.gain}')
print(results_Qi)

best 2 0.12331446334519952
Pi gain from amps data [dac units]: 11540
	Pi/2 gain from amps data [dac units]: 5770
Pi gain from avgi data [dac units]: 11625
	Pi/2 gain from avgi data [dac units]: 5812
Pi gain from avgq data [dac units]: 13099
	Pi/2 gain from avgq data [dac units]: 6549
*Set pi pulse sigma for qubit 0 to 0.02*
*Set pi pulse gain for qubit 0 to 13099*
*New pi_ge.gain
 [[13099  1914     0  1855]
 [ 2834  2504  2648  2631]
 [ 9155  9155  9155  9155]
 [ 3000  3000  3000  3000]]
*New pi_ge.gain flattened
[13099, 1914, 0, 1855, 2834, 2504, 2648, 2631, 9155, 9155, 9155, 9155, 3000, 3000, 3000, 3000]
{'Pi pulse gain': 13099}


In [26]:
if save_data: saved_files.append(amprabi.save_data())

## $T_1$

In [135]:
short_T1 = False
# short_T1 = True

In [136]:
t1 = meas.T1Experiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"t1_qubit{qubit_i}",
    config_file=config_path,
)
t1.cfg = AttrDict(deepcopy(yaml_cfg))

span = 50 if short_T1 else 400
npts = 151

t1.cfg.expt = dict(
    start=0, # wait time [us]
    step=span/npts,
    expts=npts,
    reps=reps,
    # reps=5,
    rounds=rounds,
    qTest=qubit_i,
    qZZ=None,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
    use_robust_pulses=use_robust_pulses,
)
t1.cfg.device.readout.relax_delay = span # Wait time between experiments [us]
t1.go(analyze=False, display=False, progress=True, save=False)

  0%|          | 0/10 [00:00<?, ?it/s]

In [137]:
# # TESTING
# # t1 = meas.T1Experiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_t1_qubit1.h5')
# t1.data = temp_data

t1.analyze(fit_log=False)

t1_fit, t1_fit_err = meas.fitting.get_best_fit(t1.data) #, fitter.expfunc)

t1.display(fit=True, fit_log=False)
t1_qubit = t1_fit[3]
yaml_cfg.device.readout.relax_delay[qubit_i] = np.round(4*t1_qubit)
print(f'*Set readout {qubit_i} relax delay to {yaml_cfg.device.readout.relax_delay[qubit_i]}*')
results_Qi.update({'T1':t1_qubit, 'T1 err':np.sqrt(t1_fit_err[3][3])})
print(f'qubit {qubit_i}', results_Qi)

best 1 93059.77340897589
Fit T1 amps [us]: 17.716041219403053
Fit T1 avgi [us]: 31.07377161991536
Fit T1 avgq [us]: 39.52467277526211
*Set readout 3 relax delay to 124.0*
qubit 3 {'Qubit freq': 4377.955962140133, 'T1': 31.07377161991536, 'T1 err': 1.7658704555417863}


 s:\Seb\experiements\qram_tprocv1_expts\experiments\fitting.py: 94RuntimeWarning: overflow encountered in multiply
 s:\Seb\experiements\qram_tprocv1_expts\experiments\fitting.py: 94OptimizeWarning: Covariance of the parameters could not be estimated
 s:\Seb\experiements\qram_tprocv1_expts\experiments\fitting.py: 113

In [47]:
if save_data: saved_files.append(t1.save_data())

## $T_2$ Ramsey round 1

In [233]:
# yaml_cfg.device.qubit.f_ge[qubit_i] = 145.572034953271
qubit_hwhm = 2
short_T1 = False
# short_T1 = t1_qubit < 20
print('Short T1', short_T1)

Short T1 False


In [234]:
# ramsey_freq = max((min((np.abs(np.ceil(qubit_hwhm)), 3)), 2))
ramsey_freq = 3.0
print(f'Ramsey freq [MHz]: {ramsey_freq}')

Ramsey freq [MHz]: 3.0


In [235]:
t2r = meas.RamseyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"ramsey_qubit{qubit_i}",
    config_file=config_path,
)
t2r.cfg = AttrDict(deepcopy(yaml_cfg))

# make sure nyquist freq = 0.5 * (1/step) > ramsey (signal) freq!
t2r.cfg.expt = dict(
    start=0, # wait time tau [us]
    step=soc.cycles2us(8), # [us] 
    # step=soc.cycles2us(2) if short_T1 else soc.cycles2us(10), # [us] 
    expts=200,
    ramsey_freq=ramsey_freq, # [MHz]
    reps=reps,
    rounds=1,
    qTest=qubit_i,
    checkEF=False,
    qZZ=None,
)

try:
    t2r.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

ramsey freq 3.0
Running Ramsey on Q3 


  0%|          | 0/20000 [00:00<?, ?it/s]

In [236]:
# # TESTING
# # t2r = meas.RamseyExperiment(config_file=config_path)
# temp_data, attrs = prev_data(expt_path, '00000_ramsey_qubit2.h5')
# t2r.data = temp_data

# t2r.analyze(fit=True, fit_num_sin=2)
# t2r.display(fit=True, fit_num_sin=2)

t2r.analyze(fit=True)
t2r_fit, t2r_fit_err, t2r_adjust = meas.fitting.get_best_fit(t2r.data, get_best_data_params=['f_adjust_ramsey'])

old_qubit_freq = np.reshape(t2r.cfg.device.qubit.f_ge, (4,4))[qubit_i, qubit_i]
t2r.display(fit=True)
yaml_cfg.device.qubit.f_ge = np.reshape(yaml_cfg.device.qubit.f_ge, (4,4))
yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i] = old_qubit_freq + t2r_adjust[0]
alt_qubit_freq = old_qubit_freq + t2r_adjust[1]
results_Qi.update({'T2R':t2r_fit[3], 'T2R err':np.sqrt(t2r_fit_err[3][3])})
results_Qi.update({'Qubit freq':yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
print(f'*Set qubit {qubit_i} freq to {yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i]}*')
print(f'Alt qubit freq is {alt_qubit_freq}')

print(f'*New f_ge\n', yaml_cfg.device.qubit.f_ge)
yaml_cfg.device.qubit.f_ge = yaml_cfg.device.qubit.f_ge.flatten().tolist()
print(f'*New f_ge flattened\n{yaml_cfg.device.qubit.f_ge}')

data = t2r.data
fit_axis = 'avgq'
xpts = data['xpts']
fourier = np.fft.fftshift(np.abs(np.fft.fft(data[fit_axis])))
freqs = np.fft.fftshift(np.fft.fftfreq(len(fourier), d=(xpts[1]-xpts[0])))
plt.plot(freqs, fourier)
ZZs = np.zeros((4,4))
reshaped_f_ge = np.reshape(t2r.cfg.device.qubit.f_ge, (4,4))
for row in range(4):
    ZZs[row,:] = reshaped_f_ge[row,:] - reshaped_f_ge[row,row]
for i in range(4):
    if (qubit_i+i)%4 == qubit_i: color = 'r'
    else: color = 'k'
    plt.axvline(t2r.cfg.expt.ramsey_freq - ZZs[qubit_i, (qubit_i+i)%4], color=color, linewidth=0.5, linestyle='--')
plt.xlabel('Frequency [MHz]')
plt.xlim(-6, 6)
# plt.ylim(0, 20000)
plt.title(f'Fourier Transform of Ramsey on Q{qubit_i} (Ramsey Freq: {t2r.cfg.expt.ramsey_freq})')
plt.show()

best 1 0.08977852368115806
Current pi pulse frequency: 4377.961071249494
Fit frequency from amps [MHz]: 2.966355154660412 +/- 0.004647509467397004
Suggested new pi pulse frequencies from fit amps [MHz]:
 	4377.994716094834
 	4371.994716094834
T2 Ramsey from fit amps [us]: 15.673112078244237 +/- 7.168086288391312
Current pi pulse frequency: 4377.961071249494
Fit frequency from I [MHz]: 2.963170242790809 +/- 0.00541984915077384
Suggested new pi pulse frequency from fit I [MHz]:
 	4377.997901006704
 	4371.997901006704
T2 Ramsey from fit I [us]: 9.729247008814756 +/- 3.2251268759947007
Fit frequency from Q [MHz]: 2.9678769748721088 +/- 0.004254996614594605
Suggested new pi pulse frequencies from fit Q [MHz]:
 	4377.993194274622
 	4371.993194274622
T2 Ramsey from fit Q [us]: 16.956108281336974 +/- 7.67246505340483
*Set qubit 3 freq to 4377.997901006704*
Alt qubit freq is 4371.997901006704
*New f_ge
 [[4108.73966106 4105.75887029 4110.96139089 4110.56100607]
 [3442.97914772 3448.25918515 344

In [24]:
if save_data: saved_files.append(t2r.save_data())

## Amplitude Rabi round 2

In [52]:
pi_len = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.sigma, (4,4))[qubit_i, qubit_i]
print('qubit', qubit_i, 'pi_len', pi_len)
# pi_len = 0.025

qubit 0 pi_len 0.02


In [ ]:
amprabi_2 = meas.AmplitudeRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"amp_rabi_qubit{qubit_i}",
    config_file=config_path,
)
amprabi_2.cfg = AttrDict(deepcopy(yaml_cfg))

n_pulses = 1
span = min((32000, int(1.5*np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))[qubit_i, qubit_i])))
npts = 151
    
amprabi_2.cfg.expt = dict(
    start=0, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts,
    reps=reps,
    # reps=5,
    rounds=rounds,
    sigma_test=pi_len, # gaussian sigma for pulse length - overrides config [us]
    # pulse_type='gauss',
    pulse_type='robust',
    checkEF=False, 
    n_pulses=n_pulses,
    qTest=qubit_i,
    qZZ=None,
)
amprabi_2.go(analyze=False, display=False, progress=True, save=False)

Running amp rabi on Q0 


  0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [289]:
# # TESTING
# # amprabi_2 = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_qubit0.h5')
# amprabi_2.data = temp_data

amprabi_2.analyze(fit=True)

amprabi_2_fit, amprabi_2_fit_err = meas.fitting.get_best_fit(amprabi_2.data)

p = amprabi_2_fit
if p[2] > 180: p[2] = p[2] - 360
elif p[2] < -180: p[2] = p[2] + 360
if p[2] < 0: pi_gain = (1/2 - p[2]/180)/2/p[1]
else: pi_gain = (3/2 - p[2]/180)/2/p[1]

if pi_gain > 32000:
    amprabi_2.display(fit=False)
    print(f'WARNING: Fit pi pulse gain is long ({pi_gain}), please double check!')
    pi_gain = 30000
else: amprabi_2.display(fit=True)
yaml_cfg.device.qubit.pulses.pi_ge.sigma[qubit_i] = amprabi_2.cfg.expt.sigma_test

print(f'*Set pi pulse sigma for qubit {qubit_i} to {np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.sigma, (4,4))[qubit_i, qubit_i]}*')
yaml_cfg.device.qubit.pulses.pi_ge.gain = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))
yaml_cfg.device.qubit.pulses.pi_ge.gain[qubit_i, qubit_i] = int(pi_gain)
print(f'*Set pi pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ge.gain[qubit_i, qubit_i]}*')
results_Qi.update({'Pi pulse gain':yaml_cfg.device.qubit.pulses.pi_ge.gain[qubit_i, qubit_i]})
print(f'*New pi_ge.gain\n', yaml_cfg.device.qubit.pulses.pi_ge.gain)
yaml_cfg.device.qubit.pulses.pi_ge.gain = yaml_cfg.device.qubit.pulses.pi_ge.gain.flatten().tolist()
print(f'*New pi_ge.gain flattened\n{yaml_cfg.device.qubit.pulses.pi_ge.gain}')
print(results_Qi)

best 2 0.18347979416475463
Pi gain from amps data [dac units]: 10351
	Pi/2 gain from amps data [dac units]: 5175
Pi gain from avgi data [dac units]: 9781
	Pi/2 gain from avgi data [dac units]: 4890
Pi gain from avgq data [dac units]: 6551
	Pi/2 gain from avgq data [dac units]: 3275
*Set pi pulse sigma for qubit 3 to 0.025*
*Set pi pulse gain for qubit 3 to 6551*
*New pi_ge.gain
 [[1849 1914    0 1855]
 [2834 2504 2648 2631]
 [9155 9155 9155 9155]
 [3000 3000 3000 6551]]
*New pi_ge.gain flattened
[1849, 1914, 0, 1855, 2834, 2504, 2648, 2631, 9155, 9155, 9155, 9155, 3000, 3000, 3000, 6551]
{'Qubit freq': 4380.833868595306, 'T2R': 15.590122767857144, 'T2R err': 1.4230945906229133, 'T1': 29.05419001873917, 'T1 err': 0.8144249740004452, 'Qubit EF freq': 4208.127651818629, 'T2R EF': 8.314732142857144, 'T2R EF err': 0.8397617863478003, 'T1_EF': 30.99440179962483, 'T1_EF err': 5.318884040200271, 'T2E': 20.82170758928572, 'T2E err': 1.5338494352921854, 'Pi pulse gain': 6551}


In [38]:
if save_data: saved_files.append(amprabi_2.save_data())

## $T_2$ Ramsey round 2

Check that frequency adjustment helps.

In [237]:
t2r_2 = meas.RamseyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"ramsey_qubit{qubit_i}",
    config_file=config_path,
)
t2r_2.cfg = AttrDict(deepcopy(yaml_cfg))

# make sure nyquist freq = 0.5 * (1/step) > ramsey (signal) freq!
t2r_2.cfg.expt = dict(
    start=0, # wait time tau [us]
    # step=soc.cycles2us(2) if t2r_fit[3] < 3 else soc.cycles2us(60), # [us]
    step=soc.cycles2us(150), # [us]
    expts=151,
    ramsey_freq=ramsey_freq/15, # [MHz]
    reps=reps,
    rounds=1,
    checkEF=False,
    qTest=qubit_i,
    qZZ=None,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
)
t2r_2.go(analyze=False, display=False, progress=True, save=False)

ramsey freq 0.2
Running Ramsey on Q3 


  0%|          | 0/15100 [00:00<?, ?it/s]

In [238]:
# TESTING
# t2r_2 = meas.RamseyExperiment(config_file=config_path)
# temp_data, attrs = prev_data(expt_path, '00004_ramsey_qubit2.h5')
# t2r_2.data = temp_data
# t2r_2.cfg = AttrDict(attrs['config'])

redo_t2r = True

t2r_2.analyze(fit=True)

t2r_2_fit, t2r_2_fit_err, t2r_2_adjust = meas.fitting.get_best_fit(t2r_2.data, get_best_data_params=['f_adjust_ramsey'])

t2r_2.display(fit=True)
old_qubit_freq = np.reshape(t2r_2.cfg.device.qubit.f_ge, (4,4))[qubit_i, qubit_i]
if abs(t2r_2_adjust[0]) <= ramsey_freq/10:
# if abs(t2r_2_adjust[0]) <= abs(t2r_2_adjust[0]):
    redo_t2r = False
    yaml_cfg.device.qubit.f_ge = np.reshape(yaml_cfg.device.qubit.f_ge, (4,4))
    yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i] = old_qubit_freq + t2r_2_adjust[0]
    results_Qi.update({'T2R':t2r_2_fit[3], 'T2R err':np.sqrt(t2r_2_fit_err[3][3])})
    results_Qi.update({'Qubit freq':yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
    print(f'*Set qubit {qubit_i} freq to {yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i]}*')
    print(results_Qi)
    print(f'*New f_ge\n', yaml_cfg.device.qubit.f_ge)
    yaml_cfg.device.qubit.f_ge = yaml_cfg.device.qubit.f_ge.flatten().tolist()
    print(f'*New f_ge flattened\n{yaml_cfg.device.qubit.f_ge}')

else: # redo_t2r = True
    yaml_cfg.device.qubit.f_ge[qubit_i] = alt_qubit_freq 
    print(f'*Set qubit {qubit_i} freq to {yaml_cfg.device.qubit.f_ge[qubit_i]}*')
print(f'Redo T2R: {redo_t2r}')

best 2 0.058389281243108535
Current pi pulse frequency: 4377.997901006704
Fit frequency from amps [MHz]: 0.2086622258539251 +/- 0.0011465282420421523
Suggested new pi pulse frequencies from fit amps [MHz]:
 	4377.9892387808495
 	4377.58923878085
T2 Ramsey from fit amps [us]: 22.365992936130738 +/- 3.5538318770195323
Current pi pulse frequency: 4377.997901006704
Fit frequency from I [MHz]: 0.20755509392832103 +/- 0.0018042336931341917
Suggested new pi pulse frequency from fit I [MHz]:
 	4377.990345912775
 	4377.590345912775
T2 Ramsey from fit I [us]: 19.5586368421721 +/- 4.2934700122898315
Fit frequency from Q [MHz]: 0.209234890455603 +/- 0.000944082508000557
Suggested new pi pulse frequencies from fit Q [MHz]:
 	4377.988666116248
 	4377.588666116248
T2 Ramsey from fit Q [us]: 23.339053980076482 +/- 3.1773016822867453
*Set qubit 3 freq to 4377.988666116248*
{'Qubit freq': 4377.988666116248, 'T1': 31.07377161991536, 'T1 err': 1.7658704555417863, 'T2R': 23.339053980076482, 'T2R err': 3.17

In [144]:
if save_data: saved_files.append(t2r_2.save_data())

## $\pi$, -$\pi$ experiment

In [130]:
use_robust_pulses = True

In [ ]:
piminuspi = meas.PiMinusPiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"PiMinusPiExpt{qubit_i}",
    config_file=config_path,
)

piminuspi.cfg = AttrDict(deepcopy(yaml_cfg))

span_f = 2.0
# npts_f = 50
npts_f = 30

if not use_robust_pulses:
    start_f = piminuspi.cfg.device.qubit.f_ge[qubit_i*4 + qubit_i]-span_f/2
    pulse_type = "gauss"
else:
    start_f=piminuspi.cfg.device.qubit.f_ge_robust[qubit_i*4 + qubit_i]-span_f/2,
    pulse_type = "robust"

piminuspi.cfg.expt = dict(
    start_N=0, # number gate repetitions
    step_N=4,
    expts_N=10,
    start_f=start_f,
    step_f=span_f/npts_f,
    expts_f=npts_f+1,
    reps=int(0.75*reps*rounds),
    # reps=10,
    loops=1,
    pulse_type=pulse_type,
    use_robust_pulses=use_robust_pulses,

    checkEF=False,
    qTest=qubit_i,
    qZZ=None,

    test_pi_half=False,

    post_process='threshold',
    singleshot_reps=15000,
)

# print(piminuspi.cfg)
import Pyro4.util
try:
    piminuspi.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

  0%|          | 0/2 [00:00<?, ?it/s]

Qubit  (0)
ge fidelity (%): 78.30666666666667 	 angle (deg): -96.83569591568404 	 threshold ge: 398.7212265605594


  0%|          | 0/10 [00:00<?, ?it/s]

In [132]:
# # TESTING
# temp_data, attrs = prev_data('s:\Connie\experiments\qramLL_4QR2\data\data_240617\\00000_PiMinusPiExpt0.h5')
# piminuspi.data =  temp_data
# piminuspi.cfg = AttrDict(attrs['config'])


data = piminuspi.data
piminuspi.analyze(fit=True)
piminuspi.display(fit=True)

if not use_robust_pulses:
    freq_update = "f_ge"
else:
    freq_update = "f_ge_robust"
yaml_cfg.device.qubit[freq_update] = np.reshape(yaml_cfg.device.qubit[freq_update], (4,4))
yaml_cfg.device.qubit[freq_update][qubit_i, qubit_i] = data["best_freq"]
results_Qi.update({'Qubit freq':yaml_cfg.device.qubit[freq_update][qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
print(f'*Set qubit {qubit_i} freq to {yaml_cfg.device.qubit[freq_update][qubit_i, qubit_i]}*')
print(results_Qi)
print(f'*New {freq_update}\n', yaml_cfg.device.qubit[freq_update])
yaml_cfg.device.qubit[freq_update] = yaml_cfg.device.qubit[freq_update].flatten().tolist()
print(f'*New {freq_update} flattened\n{yaml_cfg.device.qubit[freq_update]}')

Fit best freq 4111.8334058569635 which is -0.47307206386085454 away from old freq 4112.306477920824
*Set qubit 0 freq to 4111.8334058569635*
{'T1': 64.05955000251987, 'T1 err': 1.119240285515182, 'T2R': 18.248886781332068, 'T2R err': 0.7816516069106116, 'Qubit freq': 4111.8334058569635, 'T2E': 56.36959417286505, 'T2E err': 1.4283912853884384, 'Res freq': 6810.011666666666, 'Res freq (yaml)': -389.98833333333334, 'Res kappa': 0.2735895597906132, 'Res chi shift (e - g)': -0.8642384105960446, 'Pi/2 pulse gain': 7569}
*New f_ge_robust
 [[4111.83340586 4112.30647792 4112.30647792 4112.30647792]
 [3450.05408931 3450.05408931 3450.05408931 3450.05408931]
 [4763.941      4763.941      4763.941      4763.941     ]
 [4384.25034226 4384.25034226 4384.25034226 4384.25034226]]
*New f_ge_robust flattened
[4111.8334058569635, 4112.306477920824, 4112.306477920824, 4112.306477920824, 3450.0540893078382, 3450.0540893078382, 3450.0540893078382, 3450.0540893078382, 4763.941, 4763.941, 4763.941, 4763.941, 

In [ ]:
if save_data: piminuspi.save_data()

## GE error amplification

In [ ]:
npulsecalib = meas.NPulseExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"NPulseExptQ{qubit_i}",
    config_file=config_path,
)

npulsecalib.cfg = AttrDict(deepcopy(yaml_cfg))
npulsecalib.cfg.expt = dict(
    start=0, # number gate repetitions
    step=1,
    expts=10,
    reps=1000,
    loops=20,
    pulse_type='gauss',
    checkEF=False, 
    qTest=qubit_i,
    qZZ=None,
    test_pi_half=False,
    error_amp=True,

    post_process='scale',
    singleshot_reps=15000,
)

# print(npulsecalib.cfg)
import Pyro4.util
try:
    npulsecalib.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

In [392]:
# # TESTING
# temp_data, attrs = prev_data(expt_path, '00000_voltage_sweep_qubit_spectroscopy_qubit3.h5')
# data = lengthrabi.analyze(data=temp_data, fit=True)

data = npulsecalib.data
npulsecalib.analyze(fit=True, scale=None)
npulsecalib.display(fit=True, scale=None)

# npulsecalib_fit, npulsecalib_fit_err = meas.fitting.get_best_fit(npulsecalib.data)
fit_axis = 'avgi'
npulsecalib_fit = data['fit_avgi']
npulsecalib_fit_err = data['fit_err_avgi']

angle_err = npulsecalib_fit[1]
amp_ratio = (180 - angle_err) / 180
new_amp_ge = np.reshape(npulsecalib.cfg.device.qubit.pulses.pi_ge.gain, (4,4))[qubit_i, qubit_i] / amp_ratio

yaml_cfg.device.qubit.pulses.pi_ge.gain = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))
yaml_cfg.device.qubit.pulses.pi_ge.gain[qubit_i, qubit_i] = int(round(new_amp_ge))
print(f'*Set pi pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ge.gain[qubit_i, qubit_i]}*')
results_Qi.update({'Pi pulse gain':yaml_cfg.device.qubit.pulses.pi_ge.gain[qubit_i, qubit_i]})
print(f'*New pi_ge.gain\n', yaml_cfg.device.qubit.pulses.pi_ge.gain)
yaml_cfg.device.qubit.pulses.pi_ge.gain = yaml_cfg.device.qubit.pulses.pi_ge.gain.flatten().tolist()
print(f'*New pi_ge.gain flattened\n{yaml_cfg.device.qubit.pulses.pi_ge.gain}')
print(results_Qi)

NameError: name 'npulsecalib' is not defined

In [ ]:
if save_data: npulsecalib.save_data()

## $T_2$ Echo

In [40]:
# yaml_cfg.device.qubit.f_ge[qubit_i] = 145.572034953271
qubit_hwhm = 2
short_T1 = False
# short_T1 = True
# short_T1 = yaml_cfg.device.qubit.T1[qubit_i] < 20
print('Short T1', short_T1)

Short T1 False


In [41]:
ramsey_freq = max((min((np.abs(np.ceil(qubit_hwhm)), 3)), 2))
# ramsey_freq = 1
print(f'Ramsey freq [MHz]: {ramsey_freq}')

Ramsey freq [MHz]: 2.0


In [42]:
t2e = meas.RamseyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"echo_qubit{qubit_i}",
    config_file=config_path,
)
t2e.cfg = AttrDict(deepcopy(yaml_cfg))

t2e.cfg.expt = dict(
    start=0, # total wait time b/w the two pi/2 pulses [us]
    # step=soc.cycles2us(80), # make sure nyquist freq = 0.5*(1/step) > ramsey (signal) freq!
    step=soc.cycles2us(30) if short_T1 else soc.cycles2us(150), # [us] 
    expts=201,
    ramsey_freq=max((ramsey_freq/15, 0.100)), # frequency by which to advance phase [MHz]
    num_pi=1, # number of pi pulses
    echo_type='cpmg', # cp or cpmg
    reps=reps,
    # reps=10,
    rounds=rounds,
    qTest=qubit_i,
    qZZ=None,
    checkEF=False,
)
t2e.go(analyze=False, display=False, progress=True, save=False)

ramsey freq 0.13333333333333333
Running Ramsey Echo on Q1 


  0%|          | 0/10 [00:00<?, ?it/s]

In [43]:
# # TESTING
# # t2e = meas.RamseyEchoExperiment(config_file=config_path)
# temp_data, attrs = prev_data(expt_path, '00000_echo_qubit1.h5')
# t2e.data = temp_data

print(f'Qubit {qubit_i}')
t2e.analyze(fit=True)

t2e_fit, t2e_fit_err = meas.fitting.get_best_fit(t2e.data)

if not isinstance(t2e_fit, (list, np.ndarray)):
    t2e.display(fit=False)
    assert False, 'Fit failed, please manually fix!'

t2e.display(fit=True)
results_Qi.update({'T2E':t2e_fit[3], 'T2E err':np.sqrt(t2e_fit_err[3][3])})
print(results_Qi)
# print('Results updated.')

Qubit 1
best 2 0.0622798305708678
Current pi pulse frequency: 3447.2125949303563
Fit frequency from amps [MHz]: 0.1413512667059562 +/- 0.0016669960238427687
Suggested new pi pulse frequencies from fit amps [MHz]:
 	3447.2045769969836
 	3446.937910330317
T2 Ramsey from fit amps [us]: 20.82170758928572 +/- 4.3424513354334575
Current pi pulse frequency: 3447.2125949303563
Fit frequency from I [MHz]: 0.13190949055322806 +/- 0.0015833982650325178
Suggested new pi pulse frequency from fit I [MHz]:
 	3447.2140187731366
 	3446.9473521064697
T2 Ramsey from fit I [us]: 20.82170758928572 +/- 4.214340690530717
Fit frequency from Q [MHz]: 0.1342565524573538 +/- 0.0011074400647582941
Suggested new pi pulse frequencies from fit Q [MHz]:
 	3447.211671711232
 	3446.9450050445657
T2 Ramsey from fit Q [us]: 20.82170758928572 +/- 2.931229309211224
{'Qubit freq': 3447.2125949303563, 'T2R': 15.590122767857144, 'T2R err': 4.264038875225909, 'T2E': 20.82170758928572, 'T2E err': 2.931229309211224}


In [46]:
if save_data: saved_files.append(t2e.save_data())

## (Divide len) Calibrate $\pi/2$ pulse

### Amplitude rabi for $\pi/2$ pulse (divide len) (use this for protocol)

In [127]:
divide_len = True

In [128]:
pi_len = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.sigma, (4,4))[qubit_i, qubit_i]
# pi_len = 0.008
print('qubit', qubit_i, 'pi_len', pi_len)

qubit 3 pi_len 0.025


In [129]:
amprabi_3 = meas.AmplitudeRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"amp_rabi_qubit{qubit_i}",
    config_file=config_path,
)
amprabi_3.cfg = AttrDict(deepcopy(yaml_cfg))

span = min((32000, int(1.5*np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))[qubit_i, qubit_i])))
npts = 50
    
amprabi_3.cfg.expt = dict(
    start=0, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts,
    reps=reps*rounds,
    rounds=1,
    sigma_test=pi_len, # gaussian sigma for pulse length - overrides config [us]
    pulse_type="gauss" if not use_robust_pulses else "robust",
    checkEF=False, 
    n_pulses=1,
    test_pi_half=True,
    qTest=qubit_i,
    qZZ=None,

    use_robust_pulses=use_robust_pulses,
)
amprabi_3.go(analyze=False, display=False, progress=True, save=False)

Running amp rabi on Q3 


  0%|          | 0/50000 [00:00<?, ?it/s]

(4, 50, 1000)


In [130]:
# # TESTING
# # amprabi_3 = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_qubit0.h5')
# amprabi_3.data = temp_data

# amprabi_3.display(fit=False)

amprabi_3.analyze(fit=True)

amprabi_3_fit, amprabi_3_fit_err = meas.fitting.get_best_fit(amprabi_3.data)

p = amprabi_3_fit
if p[2] > 180: p[2] = p[2] - 360
elif p[2] < -180: p[2] = p[2] + 360
if p[2] < 0: pi_gain = (1/2 - p[2]/180)/2/p[1]
else: pi_gain = (3/2 - p[2]/180)/2/p[1]

if pi_gain > 32000:
    amprabi_3.display(fit=False)
    print(f'WARNING: Fit pi/2 pulse gain is long ({pi_gain}), please double check!')
    pi_gain = 30000
else: amprabi_3.display(fit=True)

yaml_cfg.device.qubit.pulses.pi_ge.half_gain = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.half_gain, (4,4))
yaml_cfg.device.qubit.pulses.pi_ge.half_gain[qubit_i, qubit_i] = int(pi_gain)
print(f'*Set pi/2 pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ge.half_gain[qubit_i, qubit_i]}*')
results_Qi.update({'Pi/2 pulse gain':yaml_cfg.device.qubit.pulses.pi_ge.half_gain[qubit_i, qubit_i]})
print(f'*New pi_ge.half_gain\n', yaml_cfg.device.qubit.pulses.pi_ge.half_gain)
yaml_cfg.device.qubit.pulses.pi_ge.half_gain = yaml_cfg.device.qubit.pulses.pi_ge.half_gain.flatten().astype(int).tolist()
print(f'*New pi_ge.half_gain flattened\n{yaml_cfg.device.qubit.pulses.pi_ge.half_gain}')
print(results_Qi)

best 0 0.03736597231018949
Pi gain from amps data [dac units]: 3120
	Pi/2 gain from amps data [dac units]: 1560
Pi gain from avgi data [dac units]: 3120
	Pi/2 gain from avgi data [dac units]: 1560
Pi gain from avgq data [dac units]: 2142
	Pi/2 gain from avgq data [dac units]: 1071
*Set pi/2 pulse gain for qubit 3 to 3120*
*New pi_ge.half_gain
 [[1850 1835 1873 1867]
 [2382 2186 2248 2289]
 [9253 9253 9253 9253]
 [2000 2000 2000 3120]]
*New pi_ge.half_gain flattened
[1850, 1835, 1873, 1867, 2382, 2186, 2248, 2289, 9253, 9253, 9253, 9253, 2000, 2000, 2000, 3120]
{'Pi pulse gain': 14360, 'Qubit freq': 4381.135527846996, 'Res freq': 6971.153333333334, 'Res freq (yaml)': -228.84666666666666, 'Res kappa': 0.20149515995222483, 'Pi/2 pulse gain': 3120}


In [ ]:
if save_data: saved_files.append(amprabi_3.save_data())

### $\pi/2$, -$\pi/2$ experiment

In [113]:
use_robust_pulses = False

In [21]:
sweep_phase = True

In [40]:
piminuspi = meas.PiMinusPiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"PiMinusPiExpt{qubit_i}",
    config_file=config_path,
)

piminuspi.cfg = AttrDict(deepcopy(yaml_cfg))

span_f = 5.0
npts_f = 30
# span_f = 2.0
# npts_f = 10

# span_f = 0.0
# npts_f = 1

start_f = piminuspi.cfg.device.qubit.f_ge[qubit_i*4 + qubit_i]-span_f/2
pulse_type = "gauss"
if use_robust_pulses:
    pulse_type = "robust"
    if not sweep_phase:
        start_f = piminuspi.cfg.device.qubit.f_ge_robust[qubit_i*4 + qubit_i]-span_f/2,
    else:
        start_f = -span_f/2

piminuspi.cfg.expt = dict(
    start_N=0, # number gate repetitions
    step_N=1,
    expts_N=10,
    # step_N=4,
    # expts_N=4,
    start_f=start_f,
    step_f=span_f/npts_f,
    expts_f=npts_f+1,
    # reps=int(0.75*reps*rounds),
    # reps=1000,
    reps=500,
    # reps=10,
    loops=1,
    pulse_type=pulse_type,
    use_robust_pulses=use_robust_pulses,

    checkEF=False,
    qTest=qubit_i,
    qZZ=None,

    # use_Y=True,

    test_pi_half=True,

    post_process='threshold',
    singleshot_reps=15000,

    sweep_phase=sweep_phase,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
)

# print(piminuspi.cfg)
import Pyro4.util
try:
    piminuspi.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

  0%|          | 0/2 [00:00<?, ?it/s]

Qubit  (3)
ge fidelity (%): 66.23234437851413 	 angle (deg): -26.387257959233988 	 threshold ge: 52.10000869145318


  0%|          | 0/10 [00:00<?, ?it/s]

In [41]:
# # TESTING
# temp_data, attrs = prev_data('s:\Connie\experiments\qramLL_4QR2\data\data_240617\\00000_PiMinusPiExpt0.h5')
# piminuspi.data =  temp_data
# piminuspi.cfg = AttrDict(attrs['config'])


data = piminuspi.data
piminuspi.analyze(fit=True)
piminuspi.display(fit=True)

if not sweep_phase:
    if not use_robust_pulses:
        freq_update = "f_ge"
    else:
        freq_update = "f_ge_robust"
    yaml_cfg.device.qubit[freq_update] = np.reshape(yaml_cfg.device.qubit[freq_update], (4,4))
    yaml_cfg.device.qubit[freq_update][qubit_i, qubit_i] = data["best_freq"]
    results_Qi.update({'Qubit freq':yaml_cfg.device.qubit[freq_update][qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
    print(f'*Set qubit {qubit_i} freq to {yaml_cfg.device.qubit[freq_update][qubit_i, qubit_i]}*')
    print(results_Qi)
    print(f'*New {freq_update}\n', yaml_cfg.device.qubit[freq_update])
    yaml_cfg.device.qubit[freq_update] = yaml_cfg.device.qubit[freq_update].flatten().tolist()
    print(f'*New {freq_update} flattened\n{yaml_cfg.device.qubit[freq_update]}')

else:
    if not use_robust_pulses:
        yaml_cfg.device.qubit.pulses.pi_ge.half_correction_phase[qubit_i*4 + qubit_i] = data["best_phase"]
        print(f'*New pi_ge.half_correction_phase \n{yaml_cfg.device.qubit.pulses.pi_ge.half_correction_phase}')
    else:
        yaml_cfg.device.qubit.pulses.pihalf_ge_robust.correction_phase[qubit_i*4 + qubit_i] = data["best_phase"]
        print(f'*New pihalf_ge_robust.correction_phase \n{yaml_cfg.device.qubit.pulses.pihalf_ge_robust.correction_phase}')

Fit best freq 1.6904291867323187 which is 1.6904291867323187 away from old freq 0
Best virtual Z (deg) 15.21386268059087
*New pihalf_ge_robust.correction_phase 
[12.831628835545475, 10.05593135477756, 0, 0, 0.11174784781734694, -0.41895542828054083, 0, 0, 0, 0, 2.9582000054694726, 0, 0, 0, 0, 15.21386268059087]


In [ ]:
if save_data: piminuspi.save_data()

### GE $\pi/2$ error amplification (divide len)

In [196]:
divide_len = True

In [197]:
npulsecalib_gehalf = meas.NPulseExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"NPulseExptQ{qubit_i}",
    config_file=config_path,
)

if not use_robust_pulses:
    pulse_type = "gauss"
else:
    pulse_type = "robust"

npulsecalib_gehalf.cfg = AttrDict(deepcopy(yaml_cfg))
npulsecalib_gehalf.cfg.expt = dict(
    start=0, # number gate repetitions
    step=1,
    expts=20,
    reps=1000,
    loops=20,
    pulse_type=pulse_type,
    test_pi_half=True,
    divide_len=divide_len,
    error_amp=True,
    use_robust_pulses=use_robust_pulses,
    use_Y=False,

    checkEF=False,
    qTest=qubit_i,
    qZZ=None,

    post_process='scale',
    singleshot_reps=15000,
)

# print(npulsecalib_gehalf.cfg)
try:
    npulsecalib_gehalf.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

  0%|          | 0/2 [00:00<?, ?it/s]

Qubit  (3)
ge fidelity (%): 81.28 	 angle (deg): 34.05335509852408 	 threshold ge: -666.1487237776781


  0%|          | 0/20 [00:00<?, ?it/s]

 s:\Connie\experiments\qram_tprocv1_expts\experiments\clifford_averager_program.py: 1742

In [202]:
# # TESTING
# temp_data, attrs = prev_data(expt_path, '00000_voltage_sweep_qubit_spectroscopy_qubit3.h5')
# data = lengthrabi.analyze(data=temp_data, fit=True)

data = npulsecalib_gehalf.data
npulsecalib_gehalf.analyze(fit=True, scale=None)
npulsecalib_gehalf.display(fit=True, scale=None)

# npulsecalib_gehalf_fit, npulsecalib_gehalf_fit_err = meas.fitting.get_best_fit(npulsecalib_gehalf.data)
fit_axis = 'avgi'
npulsecalib_gehalf_fit = data['fit_avgi']
npulsecalib_gehalf_fit_err = data['fit_err_avgi']

angle_err = npulsecalib_gehalf_fit[1]
amp_ratio = (90 - angle_err)/90
new_amp_half_ge = npulsecalib_gehalf.cfg.expt.gain / amp_ratio

if npulsecalib_gehalf.cfg.expt.use_robust_pulses:
    yaml_update = ["pihalf_ge_robust", "gain"]
else:
    yaml_update = ["pi_ge", "half_gain"]

yaml_cfg.device.qubit.pulses[yaml_update[0]][yaml_update[1]] = np.reshape(yaml_cfg.device.qubit.pulses[yaml_update[0]][yaml_update[1]], (4,4))
yaml_cfg.device.qubit.pulses[yaml_update[0]][yaml_update[1]][qubit_i, qubit_i] = int(round(new_amp_half_ge))
print(f'*Set pi/2 pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses[yaml_update[0]][yaml_update[1]][qubit_i, qubit_i]}*')
results_Qi.update({'Pi/2 pulse gain':yaml_cfg.device.qubit.pulses[yaml_update[0]][yaml_update[1]][qubit_i, qubit_i]})
print(f'*New {yaml_update[0]}.{yaml_update[1]}\n', yaml_cfg.device.qubit.pulses[yaml_update[0]][yaml_update[1]])
yaml_cfg.device.qubit.pulses[yaml_update[0]][yaml_update[1]] = yaml_cfg.device.qubit.pulses[yaml_update[0]][yaml_update[1]].flatten().astype(int).tolist()
print(f'*New {yaml_update[0]}.{yaml_update[1]} flattened\n{yaml_cfg.device.qubit.pulses[yaml_update[0]][yaml_update[1]]}')
print(results_Qi)

fitparams [None, 0.017453292519943295]
From amps: adjust amplitude to 3595 / 1.0526538911009298 = 3415.1776100310894
From avgi: adjust amplitude to 3595 / 1.004050416624822 = 3580.4974934274874
From avgq: adjust amplitude to 3595 / 0.947346260195975 = 3794.8109904991993
*Set pi/2 pulse gain for qubit 3 to 3580*
*New pihalf_ge_robust.gain
 [[ 7569  7569  7569  7569]
 [ 4982  4982  4982  4982]
 [31856 31856 31856 31856]
 [ 3595  3595  3595  3580]]
*New pihalf_ge_robust.gain flattened
[7569, 7569, 7569, 7569, 4982, 4982, 4982, 4982, 31856, 31856, 31856, 31856, 3595, 3595, 3595, 3580]
{'Res freq': 6971.126666666667, 'Res freq (yaml)': -228.87333333333333, 'Res kappa': 0.21493746441942563, 'Res chi shift (e - g)': -0.8046357615893953, 'Qubit freq': 4383.7527021237, 'Pi/2 pulse gain': 3580}


In [ ]:
if save_data: npulsecalib_gehalf.save_data()

## (Divide gain) Calibrate $\pi/2$ pulse

### Amplitude rabi for $\pi/2$ pulse (divide gain) (use this for RB)

In [ ]:
divide_len = False

In [ ]:
pi_len = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.sigma, (4,4))[qubit_i, qubit_i]
# pi_len = 0.008
print('qubit', qubit_i, 'pi_len', pi_len)

In [ ]:
amprabi_3 = meas.AmplitudeRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"amp_rabi_qubit{qubit_i}",
    config_file=config_path,
)
amprabi_3.cfg = AttrDict(deepcopy(yaml_cfg))

span = min((32000, int(1.5*0.5*np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))[qubit_i, qubit_i])))
npts = 151
    
amprabi_3.cfg.expt = dict(
    start=0, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts,
    reps=reps,
    # reps=5,
    rounds=rounds,
    sigma_test=pi_len, # gaussian sigma for pulse length - overrides config [us]
    pulse_type='gauss',
    checkEF=False, 
    n_pulses=1,
    test_pi_half=True,
    divide_len=divide_len,
    qTest=qubit_i,
    qZZ=None,
)
amprabi_3.go(analyze=False, display=False, progress=True, save=False)

In [ ]:
# # TESTING
# # amprabi_3 = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_qubit0.h5')
# amprabi_3.data = temp_data

amprabi_3.analyze(fit=True)

amprabi_3_fit, amprabi_3_fit_err = meas.fitting.get_best_fit(amprabi_3.data)

p = amprabi_3_fit
if p[2] > 180: p[2] = p[2] - 360
elif p[2] < -180: p[2] = p[2] + 360
if p[2] < 0: pi_gain = (1/2 - p[2]/180)/2/p[1]
else: pi_gain = (3/2 - p[2]/180)/2/p[1]

if pi_gain > 32000:
    amprabi_3.display(fit=False)
    print(f'WARNING: Fit pi/2 pulse gain is long ({pi_gain}), please double check!')
    pi_gain = 30000
else: amprabi_3.display(fit=True)

yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma, (4,4))
yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma[qubit_i, qubit_i] = int(pi_gain)
print(f'*Set pi/2 pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma[qubit_i, qubit_i]}*')
results_Qi.update({'Pi/2 pulse gain':yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma[qubit_i, qubit_i]})
print(f'*New pi_ge.half_gain_pi_sigma\n', yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma)
yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma = yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma.flatten().astype(int).tolist()
print(f'*New pi_ge.half_gain_pi_sigma flattened\n{yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma}')
print(results_Qi)

In [ ]:
if save_data: saved_files.append(amprabi_3.save_data())

### GE $\pi/2$ error amplification (divide gain)

In [ ]:
divide_len = False

In [ ]:
npulsecalib_gehalf = meas.NPulseExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"NPulseExptQ{qubit_i}",
    config_file=config_path,
)

npulsecalib_gehalf.cfg = AttrDict(deepcopy(yaml_cfg))
npulsecalib_gehalf.cfg.expt = dict(
    start=0, # number gate repetitions
    step=1,
    expts=10,
    reps=1000,
    loops=20,
    pulse_type='gauss',
    checkEF=False, 
    qTest=qubit_i,
    qZZ=None,
    test_pi_half=True,
    divide_len=divide_len,
    error_amp=True,

    post_process='scale',
    singleshot_reps=15000,
)

print(npulsecalib_gehalf.cfg)
import Pyro4.util
try:
    npulsecalib_gehalf.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

In [ ]:
# # TESTING
# temp_data, attrs = prev_data(expt_path, '00000_voltage_sweep_qubit_spectroscopy_qubit3.h5')
# data = lengthrabi.analyze(data=temp_data, fit=True)

data = npulsecalib_gehalf.data
npulsecalib_gehalf.analyze(fit=True, scale=None)
npulsecalib_gehalf.display(fit=True, scale=None)

# npulsecalib_gehalf_fit, npulsecalib_gehalf_fit_err = meas.fitting.get_best_fit(npulsecalib_gehalf.data)
fit_axis = 'avgi'
npulsecalib_gehalf_fit = data['fit_avgi']
npulsecalib_gehalf_fit_err = data['fit_err_avgi']

angle_err = npulsecalib_gehalf_fit[1]
amp_ratio = (90 - angle_err)/90
new_amp_half_ge = np.reshape(npulsecalib_gehalf.cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma, (4,4))[qubit_i, qubit_i] / amp_ratio

yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma, (4,4))
yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma[qubit_i, qubit_i] = int(round(new_amp_half_ge))
print(f'*Set pi/2 pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma[qubit_i, qubit_i]}*')
results_Qi.update({'Pi/2 pulse gain':yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma[qubit_i, qubit_i]})
print(f'*New pi_ge.half_gain_pi_sigma\n', yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma)
yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma = yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma.flatten().astype(int).tolist()
print(f'*New pi_ge.half_gain_pi_sigma flattened\n{yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma}')
print(results_Qi)

In [ ]:
if save_data: npulsecalib_gehalf.save_data()

## Resonator chi shift (g/e)

In [66]:
rspec_chi = meas.ResonatorSpectroscopyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"resonator_spectroscopy_chi_qubit{qubit_i}",
    config_file=config_path,
)
rspec_chi.cfg = AttrDict(deepcopy(yaml_cfg))

span = 3 # MHz
npts = 151
rspec_chi.cfg.expt = dict(
    start=rspec_chi.cfg.device.readout.frequency[qubit_i]-span/2, # MHz
    # start=rspec_chi.cfg.device.readout.frequency[qubit_i]-rspec_chi.cfg.device.readout.lo_sideband[qubit_i]*span, # MHz
    step=span/npts,
    expts=npts,
    reps=reps*rounds,
    pulse_e=True, # add ge pi pulse prior to measurement
    pulse_f=False, # add ef pi pulse prior to measurement
    qTest=qubit_i,
    qZZ=None,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
)

rspec_chi.cfg.device.readout.gain[qubit_i] /= 2
# rspec_chi.cfg.device.readout.readout_length[qubit_i] *= 2

# rspec_chi.cfg.device.readout.relax_delay = 100 # Wait time between experiments [us]
rspec_chi.go(analyze=False, display=False, progress=True, save=False)

  0%|          | 0/151 [00:00<?, ?it/s]

In [67]:
# # TESTING
# # rspec_chi = meas.ResonatorSpectroscopyExperiment(config_file=config_path,) 
# temp_data, attrs = prev_data(expt_path, '00001_resonator_spectroscopy_qubit2.h5')
# rspec_chi.data = temp_data

rspec_chi.data = rspec_chi.analyze(fit=True)
rspec_chi_fit = rspec_chi.data['fit']
if not isinstance(rspec_chi_fit, (list, np.ndarray)):
    rspec_chi.display(fit=False)
    assert False, 'Fit failed, please manually fix!'
rspec_chi.display(fit=True)
# res_freq_e = rspec_chi_fit[0] - yaml_cfg.hw.soc.dacs.readout.mixer_freq[qubit_i]
res_freq_e = rspec_chi.data['xpts'][np.argmin(rspec_chi.data['amps'])]
# res_freq_e = rspec_chi.data['xpts'][np.argmax(rspec_chi.data['amps'])]
print('Res freq e', res_freq_e)
res_freq_g = yaml_cfg.device.readout.frequency[qubit_i]
print('Res freq g', res_freq_g)
# res_freq_e = -83
# results_Qi.update({'Res chi shift (e - g)': yaml_cfg.device.readout.lo_sideband[qubit_i]*(res_freq_e - yaml_cfg.device.readout.frequency[qubit_i])})
chi_shift_eg = yaml_cfg.device.readout.lo_sideband[qubit_i]*(res_freq_e - res_freq_g)
print('Chi shift (e-g):', chi_shift_eg)
results_Qi.update({'Res chi shift (e - g)': chi_shift_eg})
# res_freq_e = 6953.5
# results_Qi.update({'Res chi shift (e - g)': res_freq_e - (5925 + yaml_cfg.device.readout.lo_sideband[qubit_i]*(yaml_cfg.hw.soc.dacs.readout.mixer_freq + yaml_cfg.device.readout.frequency[qubit_i]))})
print(f'qubit {qubit_i}', results_Qi)

electrical delay phase correction (no mux support) (deg) -0.9076642467601178
param guess [7702.358299558499, 38511.791497792496, 38511.791497792496, 0, 0.018340276016335748, 0.9086934125827005]
param bounds ([7700.888100883003, 0, 0, -6.283185307179586, -0.027510414024503622, -inf], [7703.8284982339965, 1000000000.0, 100000000.0, 6.283185307179586, 0.027510414024503622, 1.0483127995070105])
popt [ 7.70258087e+03  3.26966142e+04  4.03823664e+05 -2.88126910e+00
  2.75104140e-02 -9.44456758e-02]

Freq with minimum transmission: 501.3847896247247
Freq with maximum transmission: 501.74240551876443
From fit:
	f0: 7702.580868230989
	Qi: 32696.61417842917 	 kappa_i/2pi: 0.23557732388427507
	Qe: 403823.66418188985 	 kappa_e/2pi: 0.019074119600781
	Q0: 30247.544497751907
	kappa [MHz]: 0.2546514434850561
	phi [radians]: -2.8812691049037285
baseline 34.79599741799686
Res freq e 501.4046571743936
Res freq g 502.368233333334
Chi shift (e-g): -0.9635761589403842
qubit 2 {'Qubit EF freq': 4574.2134872

In [49]:
if save_data: saved_files.append(rspec_chi.save_data())

# Qubit ef spectroscopy

## Pulse Probe EF spectroscopy

In [15]:
qEFspec = meas.PulseProbeSpectroscopyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"qubit_EF_spectroscopy_qubit{qubit_i}",
    config_file=config_path,
)
qEFspec.cfg = AttrDict(deepcopy(yaml_cfg))

span = 100
npts = 250

qEFspec.cfg.expt = dict(
    # start=np.reshape(yaml_cfg.device.qubit.f_ef, (4,4))[qubit_i,qubit_i]-0.5*span, # resonator frequency to be mixed up [MHz]
    # start=3347-0.5*span, # resonator frequency to be mixed up [MHz]
    start=4570-0.5*span, # resonator frequency to be mixed up [MHz]
    # start=4200-0.5*span, # resonator frequency to be mixed up [MHz]
    step=span/npts, # min step ~1 Hz
    expts=npts, # Number of experiments stepping from start
    # reps=reps, # Number of averages per point
    # rounds=rounds, # Number of start to finish sweeps to average over
    reps=100000, # Number of averages per point
    rounds=1, # Number of start to finish sweeps to average over
    length=100, # ef probe constant pulse length [us]
    # gain=int(np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.gain, (4,4))[qubit_i, qubit_i]/50), # ef pulse gain
    gain=32000, # ef pulse gain
    # gain=2000,
    # pulse_type='gauss',
    pulse_type='const',
    qTest=qubit_i,
    qZZ=None,
    checkEF=True,
)
qEFspec.cfg.device.readout.relax_delay = 1 # Wait time between experiments [us]
qEFspec.go(analyze=False, display=False, progress=True, save=False)

Running pulse probe EF on Q2 


  0%|          | 0/25000000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [93]:
# # TESTING
# temp_data, attrs = prev_data(expt_path, '00000_voltage_sweep_qubit_spectroscopy_qubit3.h5')
# data = qEFspec.analyze(data=temp_data, fit=True)

best_signs = [1, 0, 0]
best_fit_err_i = np.inf
best_fit_err_q = np.inf
for sign in [1, -1]:
    try: 
        qEFspec.analyze(fit=True, signs=[1, sign, sign])
        fit_err_i = qEFspec.data['fit_err_avgi'][2][2]
        fit_err_q = qEFspec.data['fit_err_avgq'][2][2]
        if fit_err_i < best_fit_err_i:
            best_signs[1] = sign
            best_fit_err_i = fit_err_i
        if fit_err_q < best_fit_err_q:
            best_signs[2] = sign
            best_fit_err_q = fit_err_q
    except: continue
if best_fit_err_i == np.inf and best_fit_err_q == np.inf:
    qEFspec.display(fit=False)
    assert False, 'Fit failed, please manually fix!'

qEFspec.analyze(fit=True, signs=best_signs)
qEFspec.display(fit=True, signs=best_signs)

qEFspec_fit, qEFspec_fit_err = meas.fitting.get_best_fit(qEFspec.data)
yaml_cfg.device.qubit.f_ef = np.reshape(yaml_cfg.device.qubit.f_ef, (4,4))
yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i] = qEFspec_fit[2]
print(f'*Set qubit {qubit_i} f_ef to {yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i]}*')
results_Qi.update({'Qubit EF freq':yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
yaml_cfg.device.qubit.f_ef = yaml_cfg.device.qubit.f_ef.flatten().tolist()
print(results_Qi)

Found peak in amps at [MHz] 4562.353538316281, HWHM 38.19175261130648
Found peak in I at [MHz] 4573.014889281479, HWHM 0.3008438455134805
Found peak in Q at [MHz] 4573.005695603176, HWHM 0.15896427175868144
best 1 0.11335507387118181
*Set qubit 2 f_ef to 4573.014889281479*
{'Single shot g-e fidelity (%)': 81.5993155394424, 'Qubit EF freq': 4573.014889281479, 'Pi EF pulse gain': 13391, 'T2R EF': 0.7296743005857054, 'T2R EF err': 0.0705799582532272, 'T2E EF': 18.08550220693083, 'T2E EF err': 0.8401806347858306}


In [48]:
if save_data: saved_files.append(qEFspec.save_data())

## Pulse Probe EF spectroscopy round 2

In [337]:
qEFspec2 = meas.PulseProbeSpectroscopyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"qubit_EF_spectroscopy_qubit{qubit_i}",
    config_file=config_path,
)
qEFspec2.cfg = AttrDict(deepcopy(yaml_cfg))

span = 15
npts = 250

qEFspec2.cfg.expt = dict(
    start=np.reshape(yaml_cfg.device.qubit.f_ef, (4,4))[qubit_i,qubit_i]-0.5*span, # resonator frequency to be mixed up [MHz]
    step=span/npts, # min step ~1 Hz
    expts=npts, # Number of experiments stepping from start
    reps=reps, # Number of averages per point
    rounds=1, # Number of start to finish sweeps to average over
    length=1, # ef probe constant pulse length [us]
    gain=int(np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.gain, (4,4))[qubit_i, qubit_i]/80), # ef pulse gain
    pulse_type='gauss',
    qTest=qubit_i,
    qZZ=None,
    checkEF=True,
)
qEFspec2.go(analyze=False, display=False, progress=True, save=False)

Running pulse probe EF on Q1 


  0%|          | 0/25000 [00:00<?, ?it/s]

In [338]:
# # TESTING
# temp_data, attrs = prev_data(expt_path, '00000_voltage_sweep_qubit_spectroscopy_qubit3.h5')
# data = qEFspec2.analyze(data=temp_data, fit=True)

best_signs = [1, 0, 0]
best_fit_err_i = np.inf
best_fit_err_q = np.inf
for sign in [1, -1]:
    qEFspec2.analyze(fit=True, signs=[1, sign, sign])
    fit_err_i = qEFspec2.data['fit_err_avgi'][2][2]
    fit_err_q = qEFspec2.data['fit_err_avgq'][2][2]
    if fit_err_i < best_fit_err_i:
        best_signs[1] = sign
        best_fit_err_i = fit_err_i
    if fit_err_q < best_fit_err_q:
        best_signs[2] = sign
        best_fit_err_q = fit_err_q
    # except: continue
if best_fit_err_i == np.inf and best_fit_err_q == np.inf:
    qEFspec2.display(fit=False)
    assert False, 'Fit failed, please manually fix!'

qEFspec2.analyze(fit=True, signs=best_signs)
qEFspec2.display(fit=True, signs=best_signs)

qEFspec2_fit, qEFspec2_fit_err = meas.fitting.get_best_fit(qEFspec2.data)
yaml_cfg.device.qubit.f_ef = np.reshape(yaml_cfg.device.qubit.f_ef, (4,4))
yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i] = qEFspec2_fit[2]
qubit_hwhm = qEFspec2_fit[3]
print(f'*Set qubit {qubit_i} f_ef to {yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i]}*')
results_Qi.update({'Qubit EF freq':yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
yaml_cfg.device.qubit.f_ef = yaml_cfg.device.qubit.f_ef.flatten().tolist()
print(results_Qi)

Found peak in amps at [MHz] 3330.1953493264414, HWHM 0.6193900580000273
Found peak in I at [MHz] 3320.5750262121187, HWHM 0.12788182547005278
Found peak in Q at [MHz] 3330.19123274909, HWHM 0.603961222319665
best 0 0.1347470307907499
*Set qubit 1 f_ef to 3330.1953493264414*
{'Res freq': 5795.78833333, 'Res freq (yaml)': 5795.78833333, 'Res kappa': 0.13168778043737117, 'Qubit freq': 3448.25918515342, 'Pi pulse gain': 10343, 'T1': 64.74852934126194, 'T1 err': 1.1434735418617292, 'T2R': 41.037650210323974, 'T2R err': 4.606998375628519, 'Qubit EF freq': 3330.1953493264414, 'T2R EF': 8.314732142857144, 'T2R EF err': 3.106532836320881, 'Pi EF pulse gain': 1246}


In [82]:
if save_data: saved_files.append(qEFspec2.save_data())

## Amplitude EF Rabi

The EF pi pulse amplitude should be stored in <code>cfg.device.qubit.pulses.pi_ef.gain</code>.

In [18]:
pi_lenEF = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.sigma, (4,4))[qubit_i, qubit_i]
# pi_lenEF = 0.025
print('qubit', qubit_i, 'pi_lenEF', pi_lenEF)

qubit 2 pi_lenEF 0.015


In [25]:
amprabiEF = meas.AmplitudeRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix="amp_rabi_EF"+f"_qubit{qubit_i}",
    config_file=config_path,
)

amprabiEF.cfg = AttrDict(deepcopy(yaml_cfg))

span = min((32000, int(2*np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.gain, (4,4))[qubit_i, qubit_i])))
npts = 101

amprabiEF.cfg.expt = dict(
    start=1, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts,
    reps=reps*rounds,
    rounds=1,
    sigma_test=pi_lenEF, # gaussian sigma for pulse length - default from cfg [us]
    pulse_type='gauss',
    checkEF=True,
    qTest=qubit_i,
    qZZ=None,

    readout_cool=False,
    # n_init_readout=1,
    # n_trig=2,
    # init_read_wait_us=5,
    # singleshot_reps=20000,
    # use_gf_readout=True,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
    
)

# print(amprabiEF.cfg)
amprabiEF.go(analyze=False, display=False, progress=True, save=False)

Running amp rabi EF on Q2 
gain 1242
gain 1616
gain 6527
gain 1575
gain 6527


  0%|          | 0/101000 [00:00<?, ?it/s]

(4, 101, 1000)


In [26]:
# TESTING
# amprabiEF = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_EF_qubit1.h5')
# amprabiEF.data = temp_data

# amprabiEF.analyze(fit=False)
# amprabiEF.display(fit=False)

amprabiEF.analyze(fit=True, post_select=False)

amprabiEF_fit, amprabiEF_fit_err = meas.fitting.get_best_fit(amprabiEF.data)

p = amprabiEF_fit

if p[2] > 180: p[2] = p[2] - 360
elif p[2] < -180: p[2] = p[2] + 360
if p[2] < 0: pi_gain = (1/2 - p[2]/180)/2/p[1]
else: pi_gain = (3/2 - p[2]/180)/2/p[1]

if pi_gain > 32000:
    amprabiEF.display(fit=False)
    print('WARNING: Fit pi pulse gain is long, please double check!')
    pi_gain = 30000
else: amprabiEF.display(fit=True)


best 1 0.06745420193391582
Pi gain from amps data [dac units]: 10515
	Pi/2 gain from amps data [dac units]: 5257
Pi gain from avgi data [dac units]: 10740
	Pi/2 gain from avgi data [dac units]: 5370
Pi gain from avgq data [dac units]: 11328
	Pi/2 gain from avgq data [dac units]: 5664


In [36]:
print(f'*Set EF pi pulse sigma for qubit {qubit_i} to {np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.sigma, (4,4))[qubit_i, qubit_i]}*')
yaml_cfg.device.qubit.pulses.pi_ef.gain = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.gain, (4,4))
yaml_cfg.device.qubit.pulses.pi_ef.gain[qubit_i, qubit_i] = int(pi_gain)
print(f'*Set EF pi pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ef.gain[qubit_i, qubit_i]}*')
results_Qi.update({'Pi EF pulse gain':yaml_cfg.device.qubit.pulses.pi_ef.gain[qubit_i, qubit_i]})
print(f'*New pi_ef.gain\n', yaml_cfg.device.qubit.pulses.pi_ef.gain)
yaml_cfg.device.qubit.pulses.pi_ef.gain = yaml_cfg.device.qubit.pulses.pi_ef.gain.flatten().astype(int).tolist()
print(f'*New pi_ef.gain flattened\n{yaml_cfg.device.qubit.pulses.pi_ef.gain}')
print(results_Qi)

*Set EF pi pulse sigma for qubit 3 to 0.03*
*Set EF pi pulse gain for qubit 3 to 3170*
*New pi_ef.gain
 [[1245    0    0    0]
 [   0 1945    0    0]
 [   0    0 6000    0]
 [   0    0    0 3170]]
*New pi_ef.gain flattened
[1245, 0, 0, 0, 0, 1945, 0, 0, 0, 0, 6000, 0, 0, 0, 0, 3170]
{'Single shot g-e fidelity (%)': 81.5993155394424, 'Qubit EF freq': 4205.759790678515, 'Pi EF pulse gain': 3170}


In [87]:
if save_data: saved_files.append(amprabiEF.save_data())

## Length EF Rabi

In [20]:
lengthrabiEF = meas.LengthRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"length_rabiEF_qubit{qubit_i}",
    config_file=config_path,
)
lengthrabiEF.cfg = AttrDict(deepcopy(yaml_cfg))

lengthrabiEF.cfg.expt = dict(
    start=soc.cycles2us(0), # pulse length (sigma) [us]
    step=soc.cycles2us(1), # [us]
    expts=121,
    reps=reps*rounds,
    pulse_type='gauss',
    checkEF=True, 
    qTest=qubit_i,
    qZZ=None,
)
# lengthrabiEF.cfg.device.readout.relax_delay = 50 # Wait time between experiments [us]

import Pyro4.util
try:
    lengthrabiEF.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

Running length rabi EF on Q2 


  0%|          | 0/121 [00:00<?, ?it/s]

gain 1242
gain 1616
gain 6527
gain 1575
gain 1242
gain 1616
gain 6527
gain 1575
gain 6527
gain 1242
gain 1616
gain 6527
gain 1575
gain 6527
gain 1242
gain 1616
gain 6527
gain 1575
gain 6527


KeyboardInterrupt: 

In [74]:
# # TESTING
# # lengthrabiEF = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_qubit0.h5')
# lengthrabiEF.data = temp_data

lengthrabiEF.analyze(fit=True, fit_func='sin')

lengthrabiEF_fit, lengthrabiEF_fit_err = meas.fitting.get_best_fit(lengthrabiEF.data) #, fitfunc=fitter.decaysin)

lengthrabiEF.display(fit=False, fit_func='sin')
# p = lengthrabiEF_fit
# if p[2] > 180: p[2] = p[2] - 360
# elif p[2] < -180: p[2] = p[2] + 360
# if p[2] < 0: pi_lenEF = (1/2 - p[2]/180)/2/p[1]
# else: pi_lenEF = (3/2 - p[2]/180)/2/p[1]
# pi_lenEF = np.max((0.9*pi_lenEF, 0.015)) # since we run the length rabi at a < max gain

# lengthrabiEF_decay = p[3]

# print(f'Will use sigma {pi_lenEF} for amplitude rabi EF')

best 1 0.06497149654301536



In [75]:
if save_data: saved_files.append(lengthrabiEF.save_data())

## Qubit temperature

In [76]:
amprabiEF2 = meas.AmplitudeRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix="amp_rabi_EF"+f"_qubit{qubit_i}",
    config_file=config_path,
)

amprabiEF2.cfg = AttrDict(deepcopy(yaml_cfg))

reps_temp=500
rounds_temp=reps*rounds*100//500

span = np.reshape(amprabiEF2.cfg.device.qubit.pulses.pi_ef.gain, (4,4))[qubit_i, qubit_i]
npts = 4
# reps_temp = 10
# rounds_temp = 10
print('reps', reps_temp, 'rounds', rounds_temp)

amprabiEF2.cfg.expt = dict(
    start=0, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts+1,
    reps=reps_temp*rounds_temp,
    rounds=1,
    sigma_test=pi_lenEF, # gaussian sigma for pulse length - default from cfg [us]
    pulse_ge=False, # set to false when measuring qubit temp
    pulse_type='gauss',
    checkEF=True, 
    qTest=qubit_i,
    qZZ=None,


    # readout_cool=False,
    readout_cool=True,
    n_init_readout=1,
    n_trig=1,
    init_read_wait_us=2,
    singleshot_reps=10000,
    
    # use_gf_readout=True,
    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
)

# print(amprabiEF2.cfg)
amprabiEF2.go(analyze=False, display=False, progress=True, save=False)

reps 500 rounds 200
Running amp rabi EF on Q2 


  0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [30]:
post_select = False
# post_select = True
ps_threshold = None
# ps_threshold = 450

In [34]:
# TESTING
# amprabiEF_test = meas.AmplitudeRabiExperiment(config_file=config_path,)
# amprabiEF2 = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00004_amp_rabi_EF_qubit1.h5')
# amprabiEF2.data = temp_data

# amprabiEF2.analyze(fit=False)
# amprabiEF2.display(fit=False)

# fitparams = [None]*4
# fitparams[1] = 2/amprabiEF2.data['xpts'][-1]

# amprabiEF2_fit, amprabiEF2_fit_err = meas.fitting.get_best_fit(amprabiEF2.data)

# NATURAL CONSTANTS
h = 6.62607015e-34
fge = 1e6*(np.reshape(amprabiEF2.cfg.device.qubit.f_ge, (4,4))[qubit_i, qubit_i] + amprabiEF2.cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i])
kB = 1.380649e-23
################

fit_axis = 'amps'
# fit_axis = 'avgi' if post_select else 'amps'

amprabiEF.analyze(fit=False, post_select=post_select, ps_threshold=ps_threshold)
amprabiEF_data = amprabiEF.data[fit_axis]
# amprabiEF2_data = (amprabiEF2.data[fit_axis] + temp_data[fit_axis])/2
# p = amprabiEF.data[f'fit_{fit_axis}']
# efamp_normal = p[0]
efamp_normal = (max(amprabiEF.data[fit_axis]) - min(amprabiEF.data[fit_axis]))

amprabiEF2.analyze(fit=False, post_select=post_select, ps_threshold=ps_threshold)
amprabiEF2_data = amprabiEF2.data[fit_axis]
# p = amprabiEF2.data[f'fit_{fit_axis}']
# efamp_nopulse = p[0]
# fit_vals = meas.fitting.sinfunc(amprabiEF.data['xpts'], *p)
# std_dev_no_pulse = np.std(fit_vals[:10])
# efamp_nopulse = (max(fit_vals) - min(fit_vals))-2*std_dev_no_pulse
efamp_nopulse = (max(amprabiEF2.data[fit_axis]) - min(amprabiEF2.data[fit_axis]))

# amprabiEF2.display(fit=True, fit_func='sin')

print('yscale normal:', efamp_normal)
print('yscale no pulse:', efamp_nopulse)

temp = 1e3*-h*fge/(kB*np.log(efamp_nopulse/(efamp_normal+efamp_nopulse)))

print(f'qubit {qubit_i}')
print('Qubit temp [mK]:', temp)
results_Qi.update({'Qubit temp [mK]':temp})
print('Qubit E thermal population (%):', efamp_nopulse/efamp_normal*100)

amprabiEF2.display(fit=False)
results_Qi.update({'Qubit E thermal population (%)':efamp_nopulse/efamp_normal*100})
print(results_Qi)

IndexError: too many indices for array: array is 3-dimensional, but 4 were indexed

In [64]:
if save_data: saved_files.append(amprabiEF2.save_data())

## EF Ramsey Fringe
Update the ef frequency in <code>cfg.qubit.f_ef</code>.

In [23]:
qubitEF_hwhm = 2
ramsey_freq = 3

# short_T1 = False

In [27]:
ramseyEF_freq = max((min((np.abs(np.ceil(qubitEF_hwhm)), 3)), 2))

t2EFr = meas.RamseyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix="ramseyEF"+f"_qubit{qubit_i}",
    config_file=config_path,
)

t2EFr.cfg = AttrDict(deepcopy(yaml_cfg))

t2EFr.cfg.expt = dict(
    start=0, # wait time tau [us]
    # step=soc.cycles2us(10), # [us] make sure nyquist freq = 0.5 * (1/step) > ramsey (signal) freq!
    step=soc.cycles2us(4*2), # [us] make sure nyquist freq = 0.5 * (1/step) > ramsey (signal) freq!
    # step=0.05, # [us]
    expts=151,
    # ramsey_freq=ramseyEF_freq, # [MHz]
    ramsey_freq=ramsey_freq/2, # [MHz]
    checkEF=True,
    reps=reps,
    rounds=rounds,
    qTest=qubit_i,
    qZZ=None,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
    use_robust_pulses=use_robust_pulses,
)

try:
    t2EFr.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

gain 1242
gain 1616
gain 6527
gain 1575
gain 6527
gain 6527
Running Ramsey EF on Q2 


  0%|          | 0/10 [00:00<?, ?it/s]

In [28]:
t2EFr.cfg.expt

{'start': 0,
 'step': 0.018601190476190476,
 'expts': 151,
 'ramsey_freq': 1.5,
 'checkEF': True,
 'reps': 100,
 'rounds': 10,
 'qTest': 2,
 'qZZ': None,
 'full_mux_expt': False,
 'resonator_reset': None,
 'use_robust_pulses': True,
 'sigma_test': 0.015,
 'n_trig': 1,
 'n_init_readout': 0}

In [29]:
# # TESTING
# # t2EFr = meas.RamseyExperiment(config_file=config_path)
# temp_data, attrs = prev_data(expt_path, '00000_ramsey_qubit2.h5')
# t2EFr.data = temp_data

t2EFr.analyze(fit=True)
t2EFr_fit, t2EFr_fit_err, t2EFr_adjust = meas.fitting.get_best_fit(t2EFr.data, get_best_data_params=['f_adjust_ramsey'])
old_qubit_freq = np.reshape(t2EFr.cfg.device.qubit.f_ef, (4,4))[qubit_i, qubit_i]

if not isinstance(t2EFr_fit, (list, np.ndarray)):
    t2EFr.display(fit=False)
    assert False, 'Fit failed, please manually fix!'
t2EFr.display(fit=True)
yaml_cfg.device.qubit.f_ef = np.reshape(yaml_cfg.device.qubit.f_ef, (4,4))
yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i] = old_qubit_freq + t2EFr_adjust[0]
alt_qubit_freq = old_qubit_freq + t2EFr_adjust[1]
print(f'*Set qubit {qubit_i} EF freq to {yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i]}*')
results_Qi.update({'Qubit EF freq':yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
print(f'*New f_ef\n', yaml_cfg.device.qubit.f_ef)
yaml_cfg.device.qubit.f_ef = yaml_cfg.device.qubit.f_ef.flatten().tolist()
print(f'*New f_ef flattened\n{yaml_cfg.device.qubit.f_ef}')
results_Qi.update({'T2R EF':t2EFr_fit[3], 'T2R EF err':np.sqrt(t2EFr_fit_err[3][3])})
print(all_results)

best 1 0.06992566424943786
Current pi pulse frequency: 4573.166434846502
Fit frequency from amps [MHz]: 1.5207651104848348 +/- 0.014470748518399211
Suggested new pi pulse frequencies from fit amps [MHz]:
 	4573.145669736017
 	4570.145669736017
T2 Ramsey from fit amps [us]: 2.8902127460780758 +/- 0.7332728310553562
Current pi pulse frequency: 4573.166434846502
Fit frequency from I [MHz]: 1.5032313001468305 +/- 0.0038899935973831733
Suggested new pi pulse frequency from fit I [MHz]:
 	4573.163203546355
 	4570.163203546355
T2 Ramsey from fit I [us]: 12.358488123552204 +/- 3.5325760256777525
Fit frequency from Q [MHz]: 1.5140708000157648 +/- 0.004829426962807017
Suggested new pi pulse frequencies from fit Q [MHz]:
 	4573.152364046487
 	4570.152364046487
T2 Ramsey from fit Q [us]: 14.790337651356028 +/- 6.297801201004572
*Set qubit 2 EF freq to 4573.163203546355*
*New f_ef
 [[3882.6760964     0.         3882.59302316 3882.54656679]
 [3345.39573643 3347.51143391 3347.003495   3346.98172934]


In [64]:
if save_data: saved_files.append(t2EFr.save_data())

## Amplitude EF Rabi round 2

In [100]:
pi_lenEF = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.sigma, (4,4))[qubit_i, qubit_i]
# pi_lenEF = 0.030
print('qubit', qubit_i, 'pi_lenEF', pi_lenEF)

qubit 2 pi_lenEF 0.015


In [101]:
amprabiEF = meas.AmplitudeRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix="amp_rabi_EF"+f"_qubit{qubit_i}",
    config_file=config_path,
)

amprabiEF.cfg = AttrDict(deepcopy(yaml_cfg))

span = min((32000, int(4*np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.gain, (4,4))[qubit_i, qubit_i])))
npts = 101

amprabiEF.cfg.expt = dict(
    start=1, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts,
    reps=reps,
    rounds=rounds,
    sigma_test=pi_lenEF, # gaussian sigma for pulse length - default from cfg [us]
    pulse_type='gauss',
    checkEF=True, 
    qTest=qubit_i,
    qZZ=None,
    resonator_reset=resonator_reset,
    full_mux_expt=full_mux_expt,
)

# print(amprabiEF.cfg)
amprabiEF.go(analyze=False, display=False, progress=True, save=False)

Running amp rabi EF on Q2 


  0%|          | 0/10 [00:00<?, ?it/s]

(4, 101, 100)


In [102]:
# TESTING
# amprabiEF = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_EF_qubit1.h5')
# amprabiEF.data = temp_data

# amprabiEF.analyze(fit=False)
# amprabiEF.display(fit=False)

amprabiEF.analyze(fit=True)

amprabiEF_fit, amprabiEF_fit_err = meas.fitting.get_best_fit(amprabiEF.data)

p = amprabiEF_fit

if p[2] > 180: p[2] = p[2] - 360
elif p[2] < -180: p[2] = p[2] + 360
if p[2] < 0: pi_gain = (1/2 - p[2]/180)/2/p[1]
else: pi_gain = (3/2 - p[2]/180)/2/p[1]

if pi_gain > 32000:
    amprabiEF.display(fit=False)
    print('WARNING: Fit pi pulse gain is long, please double check!')
    pi_gain = 30000
else: amprabiEF.display(fit=True)

print(f'*Set EF pi pulse sigma for qubit {qubit_i} to {np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.sigma, (4,4))[qubit_i, qubit_i]}*')
yaml_cfg.device.qubit.pulses.pi_ef.gain = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.gain, (4,4))
yaml_cfg.device.qubit.pulses.pi_ef.gain[qubit_i, qubit_i] = int(pi_gain)
print(f'*Set EF pi pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ef.gain[qubit_i, qubit_i]}*')
results_Qi.update({'Pi EF pulse gain':yaml_cfg.device.qubit.pulses.pi_ef.gain[qubit_i, qubit_i]})
print(f'*New pi_ef.gain\n', yaml_cfg.device.qubit.pulses.pi_ef.gain)
yaml_cfg.device.qubit.pulses.pi_ef.gain = yaml_cfg.device.qubit.pulses.pi_ef.gain.flatten().astype(int).tolist()
print(f'*New pi_ef.gain flattened\n{yaml_cfg.device.qubit.pulses.pi_ef.gain}')
print(results_Qi)

best 1 0.011447329491976745
Pi gain from amps data [dac units]: 13292
	Pi/2 gain from amps data [dac units]: 6646
Pi gain from avgi data [dac units]: 13310
	Pi/2 gain from avgi data [dac units]: 6655
Pi gain from avgq data [dac units]: 13404
	Pi/2 gain from avgq data [dac units]: 6702
*Set EF pi pulse sigma for qubit 2 to 0.015*
*Set EF pi pulse gain for qubit 2 to 13310*
*New pi_ef.gain
 [[ 1245     0     0     0]
 [    0  1945     0     0]
 [    0     0 13310     0]
 [    0     0     0  3170]]
*New pi_ef.gain flattened
[1245, 0, 0, 0, 0, 1945, 0, 0, 0, 0, 13310, 0, 0, 0, 0, 3170]
{'Single shot g-e fidelity (%)': 81.5993155394424, 'Qubit EF freq': 4573.026110058546, 'Pi EF pulse gain': 13310, 'T2R EF': 8.278462631988553, 'T2R EF err': 3.777303874317057, 'T2E EF': 18.08550220693083, 'T2E EF err': 0.8401806347858306}


In [292]:
if save_data: saved_files.append(amprabiEF.save_data())

## EF Ramsey Fringe Round 2
Update the T2 EF but not the frequency

In [103]:
# yaml_cfg.device.qubit.f_ge[qubit_i] = 145.572034953271
# qubitEF_hwhm = 7
# short_T1 = False

In [40]:
t2EFr_2 = meas.RamseyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix="ramseyEF"+f"_qubit{qubit_i}",
    config_file=config_path,
)

t2EFr_2.cfg = AttrDict(deepcopy(yaml_cfg))

t2EFr_2.cfg.expt = dict(
    start=0, # wait time tau [us]
    step=soc.cycles2us(80), # [us] make sure nyquist freq = 0.5 * (1/step) > ramsey (signal) freq!
    # step=soc.cycles2us(40), # [us] make sure nyquist freq = 0.5 * (1/step) > ramsey (signal) freq!
    # step=0.05, # [us]
    expts=151,
    ramsey_freq=ramseyEF_freq/5, # [MHz]
    checkEF=True,
    reps=reps,
    rounds=rounds,
    qTest=qubit_i,
    qZZ=None,
)

try:
    t2EFr_2.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

gain 1242
gain 1616
gain 6527
gain 1575
gain 6527
gain 6527
Running Ramsey EF on Q2 


  0%|          | 0/10 [00:00<?, ?it/s]

In [41]:
# # TESTING
# # t2EFr_2 = meas.RamseyExperiment(config_file=config_path)
# temp_data, attrs = prev_data(expt_path, '00000_ramsey_qubit2.h5')
# t2EFr_2.data = temp_data

t2EFr_2.analyze(fit=True)
t2EFr_2_fit, t2EFr_2_fit_err, t2EFr_2_adjust = meas.fitting.get_best_fit(t2EFr_2.data, get_best_data_params=['f_adjust_ramsey'])
old_qubit_freq = np.reshape(t2EFr_2.cfg.device.qubit.f_ef, (4,4))[qubit_i, qubit_i]

if not isinstance(t2EFr_2_fit, (list, np.ndarray)):
    t2EFr_2.display(fit=False)
    assert False, 'Fit failed, please manually fix!'
t2EFr_2.display(fit=True)

if t2EFr_2_adjust[0] > ramseyEF_freq/10:
    print('WARNING: your EF frequency is not converging!')
else:
    # yaml_cfg.device.qubit.f_ef = np.reshape(yaml_cfg.device.qubit.f_ef, (4,4))
    # yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i] = old_qubit_freq + t2EFr_2_adjust[0]
    # print(f'*Set qubit {qubit_i} EF freq to {yaml_cfg.device.qubit.f_ef[qubit_i]}*')
    # print(f'*New f_ef\n', yaml_cfg.device.qubit.f_ef)
    # yaml_cfg.device.qubit.f_ef = yaml_cfg.device.qubit.f_ef.flatten().tolist()
    # print(f'*New f_ef flattened\n{yaml_cfg.device.qubit.f_ef}')
    results_Qi.update({'T2R EF':t2EFr_2_fit[3], 'T2R EF err':np.sqrt(t2EFr_2_fit_err[3][3])})
    # results_Qi.update({'Qubit EF freq':yaml_cfg.device.qubit.f_ef[qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
    print(results_Qi)

best 2 0.06676006672388451
Current pi pulse frequency: 4573.0141763510965
Fit frequency from amps [MHz]: 0.3875217144710908 +/- 0.004848613832952837
Suggested new pi pulse frequencies from fit amps [MHz]:
 	4573.026654636626
 	4572.226654636625
T2 Ramsey from fit amps [us]: 8.314732142857144 +/- 2.1993663138593638
Current pi pulse frequency: 4573.0141763510965
Fit frequency from I [MHz]: 0.3962873180664751 +/- 0.0038174530467537443
Suggested new pi pulse frequency from fit I [MHz]:
 	4573.01788903303
 	4572.21788903303
T2 Ramsey from fit I [us]: 8.314732142857144 +/- 1.6057627388074527
Fit frequency from Q [MHz]: 0.4079148605493975 +/- 0.003053385750103629
Suggested new pi pulse frequencies from fit Q [MHz]:
 	4573.006261490547
 	4572.206261490547
T2 Ramsey from fit Q [us]: 8.314732142857144 +/- 1.2761023480133218
{'Qubit EF freq': 4573.0141763510965, 'T2R EF': 8.314732142857144, 'T2R EF err': 1.2761023480133218}


In [ ]:
if save_data: saved_files.append(t2EFr.save_data())

## EF $T_2$ Echo

In [1]:
# yaml_cfg.device.qubit.f_ge[qubit_i] = 145.572034953271
qubit_hwhm = 2
short_T1 = False
# short_T1 = True
# short_T1 = yaml_cfg.device.qubit.T1[qubit_i] < 20
print('Short T1', short_T1)

Short T1 False


In [2]:
t2EFe = meas.RamseyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"echo_qubit{qubit_i}",
    config_file=config_path,
)
t2EFe.cfg = AttrDict(deepcopy(yaml_cfg))

t2EFe.cfg.expt = dict(
    start=0, # total wait time b/w the two pi/2 pulses [us]
    # step=soc.cycles2us(80), # make sure nyquist freq = 0.5*(1/step) > ramsey (signal) freq!
    step=soc.cycles2us(30) if short_T1 else soc.cycles2us(80), # [us] 
    expts=201,
    ramsey_freq=ramseyEF_freq/10, # frequency by which to advance phase [MHz]
    num_pi=1, # number of pi pulses
    echo_type='cpmg', # cp or cpmg
    reps=reps,
    # reps=10,
    rounds=rounds,
    qTest=qubit_i,
    qZZ=None,
    checkEF=True,
)
t2EFe.go(analyze=False, display=False, progress=True, save=False)

NameError: name 'meas' is not defined

In [66]:
# # TESTING
# # t2e = meas.RamseyEchoExperiment(config_file=config_path)
# temp_data, attrs = prev_data(expt_path, '00000_echo_qubit1.h5')
# t2e.data = temp_data

print(f'Qubit {qubit_i}')
t2EFe.analyze(fit=True)

t2EFe_fit, t2EFe_fit_err = meas.fitting.get_best_fit(t2EFe.data)

if not isinstance(t2EFe_fit, (list, np.ndarray)):
    t2EFe.display(fit=False)
    assert False, 'Fit failed, please manually fix!'

t2EFe.display(fit=True)
results_Qi.update({'T2E EF':t2EFe_fit[3], 'T2E EF err':np.sqrt(t2EFe_fit_err[3][3])})
print(results_Qi)
# print('Results updated.')

Qubit 2
best 2 0.020396588112144686
Current pi pulse frequency: 4572.992628188545
Fit frequency from amps [MHz]: 0.19973560482332373 +/- 0.00043259641175618003
Suggested new pi pulse frequencies from fit amps [MHz]:
 	4572.992892583721
 	4572.592892583722
T2 Ramsey from fit amps [us]: 20.61495980530881 +/- 1.1541883851538433
Current pi pulse frequency: 4572.992628188545
Fit frequency from I [MHz]: 0.19937379220565327 +/- 0.0004385508092021688
Suggested new pi pulse frequency from fit I [MHz]:
 	4572.993254396339
 	4572.593254396339
T2 Ramsey from fit I [us]: 18.773885363354914 +/- 0.9701854385168508
Fit frequency from Q [MHz]: 0.1992007738696078 +/- 0.00041072211936617355
Suggested new pi pulse frequencies from fit Q [MHz]:
 	4572.993427414675
 	4572.593427414675
T2 Ramsey from fit Q [us]: 18.08550220693083 +/- 0.8401806347858306
{'Single shot g-e fidelity (%)': 81.5993155394424, 'Qubit EF freq': 4572.992628188545, 'Pi EF pulse gain': 13391, 'T2R EF': 8.314732142857144, 'T2R EF err': 0

In [ ]:
if save_data: saved_files.append(t2e.save_data())

## EF error amplification

In [42]:
npulsecalibEF = meas.NPulseExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"NPulseExptEFQ{qubit_i}",
    config_file=config_path,
)

npulsecalibEF.cfg = AttrDict(deepcopy(yaml_cfg))
npulsecalibEF.cfg.expt = dict(
    start=0, # number gate repetitions
    step=1,
    expts=10,
    reps=1000,
    loops=10,
    pulse_type='gauss',
    checkEF=True, 
    qTest=qubit_i,
    qZZ=None,
    test_pi_half=False,
    error_amp=True,
    resonator_reset=resonator_reset,
    full_mux_expt=full_mux_expt,

    post_process='scale',
    singleshot_reps=15000,
    # singleshot_reps=1000,
)

print(npulsecalibEF.cfg)
try:
    npulsecalibEF.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

{'device': {'readout': {'trig_offset': 150, 'lo_sideband': [1, 1, 1, 1], 'frequency': [-389.975, 5795.761666663333, 502.368233333334, -228.9], 'gain': [0.4, 0.5, 0.3, 0.2], 'readout_length': [5, 20, 10, 8], 'frequency_ef': [-391.91408405, 5795.76166666, 500.44439227, -230.30732892], 'gain_ef': [0.05, 0.6, 0.4, 0.25], 'readout_length_ef': [1, 20, 10, 8], 'relax_delay': [450, 450, 450, 450]}, 'qubit': {'f_ge': [4108.6352143528575, 4103.243553024718, 4108.491689964749, 4108.066288400736, 3442.8688097468294, 3448.2597196742877, 3447.149851461427, 3446.943321983037, 4759.488589798401, 4758.50698515513, 4759.617265888647, 4759.085816492836, 4377.407821334912, 4376.654922763285, 4377.445747133665, 4377.972703369784], 'f_ef': [3882.584812861004, 0.0, 3882.593023159515, 3882.5465667921762, 3345.3957364303, 3347.8123422280078, 3347.003494996075, 3346.9817293387923, 4569.728581643396, 4568.311187621723, 4573.709286177009, 0.0, 4206.708200302393, 4202.366145184353, 4205.110372632136, 4205.16285596

  0%|          | 0/2 [00:00<?, ?it/s]

Qubit  (2)
g unrotated averages:
I 12.254994596354166 +/- 17.173254528351915 	 Q -2.334147482638889 +/- 16.480928939024064 	 Amp 23.51503743869217 +/- 13.008375804573193
Rotated (theta=0.46690709417052384):
I 11.993940101494733 +/- 17.46113359682598 	 Q 3.4319874517742344 +/- 16.17561446444973 	 Amp 12.475301079628473 +/- 13.008375804573193
e unrotated averages:
I 42.26288747829861 +/- 22.716562865942937 	 Q -17.46055679253472 +/- 17.91547551558382 	 Amp 49.898340920490334 +/- 20.932863308276794
Rotated (theta=0.46690709417052384):
I 45.598729846778895 +/- 23.750188339419836 	 Q 3.4319874517742326 +/- 16.520745904283718 	 Amp 45.72770168627176 +/- 20.932863308276794
ge fidelity (%): 62.35333333333335 	 angle (deg): 26.751805920688298 	 threshold ge: 30.544836886144537


  0%|          | 0/10 [00:00<?, ?it/s]

 s:\Seb\experiements\qram_tprocv1_expts\experiments\clifford_averager_program.py: 1877

In [43]:
# # TESTING
# temp_data, attrs = prev_data(expt_path, '00000_voltage_sweep_qubit_spectroscopy_qubit3.h5')
# data = lengthrabi.analyze(data=temp_data, fit=True)

data = npulsecalibEF.data
npulsecalibEF.analyze(fit=True)
npulsecalibEF.display(fit=True)

# npulsecalib_fit, npulsecalib_fit_err = meas.fitting.get_best_fit(npulsecalibEF.data)
fit_axis = 'avgi'
npulsecalibEF_fit = data[f'fit_{fit_axis}']
npulsecalibEF_fit_err = data[f'fit_err_{fit_axis}']

angle_err = npulsecalibEF_fit[1]
amp_ratio = (180 - angle_err) / 180
new_amp_ef = np.reshape(npulsecalibEF.cfg.device.qubit.pulses.pi_ef.gain, (4,4))[qubit_i, qubit_i] / amp_ratio

yaml_cfg.device.qubit.pulses.pi_ef.gain = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.gain, (4,4))
yaml_cfg.device.qubit.pulses.pi_ef.gain[qubit_i, qubit_i] = int(round(new_amp_ef))
print(f'*Set EF pi pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ef.gain[qubit_i, qubit_i]}*')
results_Qi.update({'Pi EF pulse gain':yaml_cfg.device.qubit.pulses.pi_ef.gain[qubit_i, qubit_i]})
print(f'*New pi_ef.gain\n', yaml_cfg.device.qubit.pulses.pi_ef.gain)
yaml_cfg.device.qubit.pulses.pi_ef.gain = yaml_cfg.device.qubit.pulses.pi_ef.gain.flatten().tolist()
print(f'*New pi_ef.gain flattened\n{yaml_cfg.device.qubit.pulses.pi_ef.gain}')
print(results_Qi)

fitparams [None, 0.017453292519943295]
From amps: adjust amplitude to 6501 / 0.9974906391261065 = 6517.354394117898
From avgi: adjust amplitude to 6501 / 0.9943489561683252 = 6537.946220662094
From avgq: adjust amplitude to 6501 / 0.9994160582687442 = 6504.798423252744
*Set EF pi pulse gain for qubit 2 to 13316*
*New pi_ef.gain
 [[ 1245     0     0     0]
 [    0  1945     0     0]
 [    0     0 13316     0]
 [    0     0     0  1500]]
*New pi_ef.gain flattened
[1245, 0, 0, 0, 0, 1945, 0, 0, 0, 0, 13316, 0, 0, 0, 0, 1500]
{'Qubit EF freq': 4573.709286177009, 'T2R EF': 15.233515280933435, 'T2R EF err': 1.6586265411281977, 'Pi EF pulse gain': 13316, 'T2E EF': 21.632448119150048, 'T2E EF err': 1.6481093691006576}


In [ ]:
if save_data: npulsecalibEF.save_data()

## (Divide len) Calibrate EF $\pi/2$ pulse
this is mostly only for RB

### Amplitude rabi for EF $\pi/2$ pulse (divide len)

In [38]:
pi_lenEF = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.sigma, (4,4))[qubit_i, qubit_i]
# pi_len = 0.008
print('qubit', qubit_i, 'pi_lenEF', pi_lenEF)

qubit 3 pi_lenEF 0.03


In [39]:
divide_len = True

In [40]:
amprabi_4 = meas.AmplitudeRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"amp_rabi_qubit{qubit_i}",
    config_file=config_path,
)
amprabi_4.cfg = AttrDict(deepcopy(yaml_cfg))

n_pulses = 1
span = min((32000, int(1.5*np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.gain, (4,4))[qubit_i, qubit_i])))
npts = 151
    
amprabi_4.cfg.expt = dict(
    start=0, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts,
    reps=reps,
    # reps=5,
    rounds=rounds,
    sigma_test=pi_lenEF, # gaussian sigma for pulse length - overrides config [us]
    pulse_type='gauss' if not use_robust_pulses else 'robust', # EF pulse will always be gauss, need this because will throw an error otherwise even if only using robust for the g
    n_pulses=n_pulses,
    test_pi_half=True,
    divide_len=divide_len,
    qTest=qubit_i,
    qZZ=None,
    checkEF=True, 

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
    use_robust_pulses=use_robust_pulses,
)
amprabi_4.go(analyze=False, display=False, progress=True, save=False)

Running amp rabi EF on Q3 


  0%|          | 0/10 [00:00<?, ?it/s]

(4, 151, 100)


In [41]:
# # TESTING
# # amprabi_4 = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_qubit0.h5')
# amprabi_4.data = temp_data

amprabi_4.analyze(fit=True)

amprabi_4_fit, amprabi_4_fit_err = meas.fitting.get_best_fit(amprabi_4.data)

p = amprabi_4_fit
if p[2] > 180: p[2] = p[2] - 360
elif p[2] < -180: p[2] = p[2] + 360
if p[2] < 0: pi_gain = (1/2 - p[2]/180)/2/p[1]
else: pi_gain = (3/2 - p[2]/180)/2/p[1]

if pi_gain > 32000:
    amprabi_4.display(fit=False)
    print(f'WARNING: Fit pi/2 pulse gain is long ({pi_gain}), please double check!')
    pi_gain = 30000
else: amprabi_4.display(fit=True)

yaml_cfg.device.qubit.pulses.pi_ef.half_gain = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.half_gain, (4,4))
yaml_cfg.device.qubit.pulses.pi_ef.half_gain[qubit_i, qubit_i] = int(pi_gain)
print(f'*Set pi/2 EF pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ef.half_gain[qubit_i, qubit_i]}*')
results_Qi.update({'Pi/2 EF pulse gain':yaml_cfg.device.qubit.pulses.pi_ef.half_gain[qubit_i, qubit_i]})
print(f'*New pi_ef.half_gain\n', yaml_cfg.device.qubit.pulses.pi_ef.half_gain)
yaml_cfg.device.qubit.pulses.pi_ef.half_gain = yaml_cfg.device.qubit.pulses.pi_ef.half_gain.flatten().astype(int).tolist()
print(f'*New pi_ef.half_gain flattened\n{yaml_cfg.device.qubit.pulses.pi_ef.half_gain}')
print(results_Qi)

best 1 0.015136708401338026
Pi gain from amps data [dac units]: 1874
	Pi/2 gain from amps data [dac units]: 937
Pi gain from avgi data [dac units]: 1750
	Pi/2 gain from avgi data [dac units]: 875
Pi gain from avgq data [dac units]: 1828
	Pi/2 gain from avgq data [dac units]: 914
*Set pi/2 EF pulse gain for qubit 3 to 1750*
*New pi_ef.half_gain
 [[1360    0    0    0]
 [   0 1617    0    0]
 [   0 5905 6516    0]
 [1464 1651    0 1750]]
*New pi_ef.half_gain flattened
[1360, 0, 0, 0, 0, 1617, 0, 0, 0, 5905, 6516, 0, 1464, 1651, 0, 1750]
{'Qubit EF freq': 4205.705165183835, 'T2R EF': 8.314732142857144, 'T2R EF err': 3.807735986404127, 'Pi/2 EF pulse gain': 1750}


In [ ]:
if save_data: saved_files.append(amprabi_4.save_data())

### EF $\pi/2$, -$\pi/2$ experiment

In [52]:
piminuspi_EF = meas.PiMinusPiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"PiMinusPi_EFExpt{qubit_i}",
    config_file=config_path,
)

span_f = 2.0
npts_f = 35
# npts_f = 50

# span_f = 0.0
# npts_f = 1

piminuspi_EF.cfg = AttrDict(deepcopy(yaml_cfg))
piminuspi_EF.cfg.expt = dict(
    start_N=0, # number gate repetitions
    # step_N=5,
    # step_N=2,
    # expts_N=15,
    step_N=4,
    expts_N=10,
    start_f=piminuspi_EF.cfg.device.qubit.f_ef[qubit_i*4 + qubit_i]-span_f/2,
    step_f=span_f/npts_f,
    expts_f=npts_f+1,
    reps=int(0.75*reps*rounds),
    loops=1,
    # reps=int(2*reps*rounds),
    # loops=5,
    pulse_type='gauss',

    checkEF=True,
    qTest=qubit_i,
    qZZ=None,

    # use_Y=True,

    test_pi_half=True,
    divide_len=divide_len,

    post_process='threshold',
    singleshot_reps=15000,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
)

print(piminuspi_EF.cfg)
try:
    piminuspi_EF.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

{'device': {'readout': {'trig_offset': 150, 'lo_sideband': [1, 1, 1, 1], 'frequency': [-389.975, 5795.761666663333, 502.368233333334, -228.9], 'gain': [0.4, 0.5, 0.3, 0.2], 'readout_length': [5, 20, 10, 8], 'frequency_ef': [-391.91408405, 5795.76166666, 500.44439227, -230.30732892], 'gain_ef': [0.05, 0.6, 0.4, 0.25], 'readout_length_ef': [1, 20, 10, 8], 'relax_delay': [450, 450, 450, 450]}, 'qubit': {'f_ge': [4108.6352143528575, 4103.243553024718, 4108.491689964749, 4108.066288400736, 3442.8688097468294, 3448.2597196742877, 3447.149851461427, 3446.943321983037, 4759.488589798401, 4758.50698515513, 4759.617265888647, 4759.085816492836, 4377.407821334912, 4376.654922763285, 4377.445747133665, 4377.972703369784], 'f_ef': [3882.584812861004, 0.0, 3882.593023159515, 3882.5465667921762, 3345.3957364303, 3347.8123422280078, 3347.003494996075, 3346.9817293387923, 4569.728581643396, 4568.311187621723, 4574.269327358868, 0.0, 4206.708200302393, 4202.366145184353, 4205.110372632136, 4205.16285596

  0%|          | 0/2 [00:00<?, ?it/s]

Qubit  (2)
ge fidelity (%): 63.85333333333332 	 angle (deg): 28.354092207223516 	 threshold ge: 32.430638757043745


  0%|          | 0/10 [00:00<?, ?it/s]

In [53]:
# # TESTING
# temp_data, attrs = prev_data('s:\Connie\experiments\qramLL_4QR2\data\data_240617\\00000_PiMinusPi_EFExpt0.h5')
# piminuspi_EF.data =  temp_data
# piminuspi_EF.cfg = AttrDict(attrs['config'])


data = piminuspi_EF.data

# piminuspi_EF.analyze(fit=False)
# plt.figure()
# plt.plot(data['cycle_sweep'], data['avgi'][:,0], '.-')
# plt.xlabel('N Cycles $(\pi/2, -\pi/2)^N$')
# plt.ylabel('Avg I')
# plt.show()

piminuspi_EF.analyze(fit=True)
piminuspi_EF.display(fit=True)

yaml_cfg.device.qubit.f_ef = np.reshape(yaml_cfg.device.qubit.f_ef, (4,4))
yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i] = data["best_freq"]
results_Qi.update({'Qubit EF freq':yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
print(f'*Set qubit {qubit_i} EF freq to {yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i]}*')
print(results_Qi)
print(f'*New f_ef\n', yaml_cfg.device.qubit.f_ef)
yaml_cfg.device.qubit.f_ef = yaml_cfg.device.qubit.f_ef.flatten().tolist()
print(f'*New f_ef flattened\n{yaml_cfg.device.qubit.f_ef}')

Fit best freq 4574.2134872558845 which is -0.0558401029838933 away from old freq 4574.269327358868
*Set qubit 2 EF freq to 4574.2134872558845*
{'Qubit EF freq': 4574.2134872558845, 'T2R EF': 15.233515280933435, 'T2R EF err': 1.6586265411281977, 'Pi EF pulse gain': 13316, 'T2E EF': 21.632448119150048, 'T2E EF err': 1.6481093691006576, 'Pi/2 EF pulse gain': 30000}
*New f_ef
 [[3882.58481286    0.         3882.59302316 3882.54656679]
 [3345.39573643 3347.81234223 3347.003495   3346.98172934]
 [4569.72858164 4568.31118762 4574.21348726    0.        ]
 [4206.7082003  4202.36614518 4205.11037263 4205.16285596]]
*New f_ef flattened
[3882.584812861004, 0.0, 3882.593023159515, 3882.5465667921762, 3345.3957364303, 3347.8123422280078, 3347.003494996075, 3346.9817293387923, 4569.728581643396, 4568.311187621723, 4574.2134872558845, 0.0, 4206.708200302393, 4202.366145184353, 4205.110372632136, 4205.162855964244]


In [ ]:
if save_data: piminuspi.save_data()

### EF $\pi/2$ error amplification (divide len)

In [115]:
divide_len = True

In [138]:
npulsecalib_efhalf = meas.NPulseExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"NPulseExptEF_half_Q{qubit_i}",
    config_file=config_path,
)

npulsecalib_efhalf.cfg = AttrDict(deepcopy(yaml_cfg))
npulsecalib_efhalf.cfg.expt = dict(
    start=0, # number gate repetitions
    step=1,
    expts=10,
    reps=1000,
    # loops=20,
    loops=5,
    pulse_type='gauss',
    test_pi_half=True,
    divide_len=divide_len,
    error_amp=True,

    checkEF=True,
    qTest=qubit_i,
    qZZ=None,

    # readout_ge=False, # whether to readout at the g/e set point (True) or e/f set point (False)
    readout_ge=True, # whether to readout at the g/e set point (True) or e/f set point (False)

    post_process='scale',
    # post_process=None,
    singleshot_reps=15000,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,

    use_robust_pulses=True,
)

print(npulsecalib_efhalf.cfg)
import Pyro4.util
try:
    npulsecalib_efhalf.acquire(progress=True, debug=True)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

{'device': {'readout': {'trig_offset': 150, 'lo_sideband': [1, 1, 1, 1], 'frequency': [-439.97500000000036, 0.0, 452.3682333333336, -278.89999999999964], 'gain': [0.17, 0.035, 0.25, 0.17], 'kappa_ext': [0.157, 0.065, 0.182, 0.104], 'kerr': [-0.0045, 0, 0.008, -0.003], 't_rise_reset': [0.5, 1, 0.5, 0.5], 'readout_length': [4, 4, 4, 4], 'frequency_ef': [-439.97500000000036, 0.0, 450.44439227373095, -280.30732891832196], 'readout_length_ef': 4, 'relax_delay': [450, 450, 450, 450]}, 'qubit': {'f_ge': [4111.097866562327, 4105.838904759637, 4110.958153299016, 4110.546407850519, 3442.8755721075536, 3448.5530242338505, 3447.060317317807, 3446.4929792801067, 4762.761504132457, 4761.823567235975, 4762.909473544484, 4762.369813422167, 4380.698604555338, 4379.604868123916, 4380.717837106693, 4381.248121020163], 'f_ge_robust': [4114.313815659153, 4108.147701523904, 4114.310372455052, 4114.310372455052, 3443.0032303441935, 3448.345849629168, 0, 0, 4763.941, 4763.941, 4763.941, 4763.941, 4384.2503422

  0%|          | 0/2 [00:00<?, ?it/s]

Qubit  (2)
g unrotated averages:
Amps 118.20467411805085 +/- 57.38787388991945
e unrotated averages:
Amps 279.91002155643014 +/- 67.09894931806625
ge fidelity (%): 81.78613640909393 	 angle (deg): 0.0 	 threshold ge: 196.18055470399742
thresholds=[0, 0, 196.18055470399742, 0],
angles=[0, 0, 0.0, 0],
ge_avgs=[array([0., 0., 0., 0.]), array([0., 0., 0., 0.]), array([118.20467412,   0.        , 279.91002156,   0.        ]), array([0., 0., 0., 0.])],
counts_calib=[[13720.0, 1280.0], [1456.0, 13544.0]]


  0%|          | 0/5 [00:00<?, ?it/s]

 s:\Connie\experiments\qram_tprocv1_expts\experiments\clifford_averager_program.py: 1822

In [144]:
# # TESTING
# temp_data, attrs = prev_data(expt_path, '00000_voltage_sweep_qubit_spectroscopy_qubit3.h5')
# data = lengthrabi.analyze(data=temp_data, fit=True)

data = npulsecalib_efhalf.data
npulsecalib_efhalf.analyze(fit=True, scale=None)
npulsecalib_efhalf.display(fit=True, scale=None)

# npulsecalib_efhalf_fit, npulsecalib_efhalf_fit_err = meas.fitting.get_best_fit(npulsecalib_efhalf.data)
fit_axis = 'avgi'
npulsecalib_efhalf_fit = data[f'fit_{fit_axis}']
npulsecalib_efhalf_fit_err = data[f'fit_err_{fit_axis}']

angle_err = npulsecalib_efhalf_fit[1]
amp_ratio = (90 - angle_err)/90
new_amp_half_ef = np.reshape(npulsecalib_efhalf.cfg.device.qubit.pulses.pi_ef.half_gain, (4,4))[qubit_i, qubit_i] / amp_ratio

yaml_cfg.device.qubit.pulses.pi_ef.half_gain = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.half_gain, (4,4))
yaml_cfg.device.qubit.pulses.pi_ef.half_gain[qubit_i, qubit_i] = int(round(new_amp_half_ef))
print(f'*Set pi/2 EF pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ef.half_gain[qubit_i, qubit_i]}*')
results_Qi.update({'Pi/2 EF pulse gain':yaml_cfg.device.qubit.pulses.pi_ef.half_gain[qubit_i, qubit_i]})
print(f'*New pi_ef.half_gain\n', yaml_cfg.device.qubit.pulses.pi_ef.half_gain)
yaml_cfg.device.qubit.pulses.pi_ef.half_gain = yaml_cfg.device.qubit.pulses.pi_ef.half_gain.flatten().astype(int).tolist()
print(f'*New pi_ef.half_gain flattened\n{yaml_cfg.device.qubit.pulses.pi_ef.half_gain}')
print(results_Qi)

fitparams [None, 0.017453292519943295]
Attempted to init fitparam 0 to 0.0, which is out of bounds 0.0 to 0.0. Instead init to 0.0
From amps: adjust amplitude to 6378 / 0.9989780638148239 = 6384.524576690065
From avgi: adjust amplitude to 6378 / 0.9989780638148239 = 6384.524576690065
From avgq: adjust amplitude to 6378 / 1.000193925472444 = 6376.76338314826
*Set pi/2 EF pulse gain for qubit 2 to 6385*
*New pi_ef.half_gain
 [[1261    0    0    0]
 [   0 1941    0    0]
 [   0    0 6385    0]
 [ 340  340    0  320]]
*New pi_ef.half_gain flattened
[1261, 0, 0, 0, 0, 1941, 0, 0, 0, 0, 6385, 0, 340, 340, 0, 320]
{'Res freq': 7702.328233333334, 'Res freq (yaml)': 452.32823333333357, 'Res kappa': 0.2819786993431654, 'Res chi shift (e - g)': -0.943708609271539, 'Res chi shift (f - g)': -1.9172185430463742, 'Res freq ef (yaml)': 450.4510147902872, 'Pi/2 EF pulse gain': 6385, 'Qubit EF freq': 4573.168610947125}


In [ ]:
if save_data: npulsecalib_efhalf.save_data()

## (Divide gain) Calibrate EF $\pi/2$ pulse

### Amplitude rabi for EF $\pi/2$ pulse (divide gain) (use this for RB)

In [54]:
pi_lenEF = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.sigma, (4,4))[qubit_i, qubit_i]
# pi_len = 0.008
print('qubit', qubit_i, 'pi_lenEF', pi_lenEF)

qubit 2 pi_lenEF 0.015


In [55]:
divide_len = False

In [ ]:
amprabi_4 = meas.AmplitudeRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"amp_rabi_qubit{qubit_i}",
    config_file=config_path,
)
amprabi_4.cfg = AttrDict(deepcopy(yaml_cfg))

n_pulses = 1
span = min((32000, int(1.5*0.5*np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.gain, (4,4))[qubit_i, qubit_i])))
npts = 151
    
amprabi_4.cfg.expt = dict(
    start=0, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts,
    reps=reps,
    # reps=5,
    rounds=rounds,
    sigma_test=pi_lenEF, # gaussian sigma for pulse length - overrides config [us]
    pulse_type='gauss',
    checkEF=True, 
    n_pulses=n_pulses,
    test_pi_half=True,
    divide_len=divide_len,
    qTest=qubit_i,
    qZZ=None,
)
amprabi_4.go(analyze=False, display=False, progress=True, save=False)

In [ ]:
# # TESTING
# # amprabi_4 = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_qubit0.h5')
# amprabi_4.data = temp_data

amprabi_4.analyze(fit=True)

amprabi_4_fit, amprabi_4_fit_err = meas.fitting.get_best_fit(amprabi_4.data)

p = amprabi_4_fit
if p[2] > 180: p[2] = p[2] - 360
elif p[2] < -180: p[2] = p[2] + 360
if p[2] < 0: pi_gain = (1/2 - p[2]/180)/2/p[1]
else: pi_gain = (3/2 - p[2]/180)/2/p[1]

if pi_gain > 32000:
    amprabi_4.display(fit=False)
    print(f'WARNING: Fit pi/2 pulse gain is long ({pi_gain}), please double check!')
    pi_gain = 30000
else: amprabi_4.display(fit=True)

yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma, (4,4))
yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma[qubit_i, qubit_i] = int(pi_gain)
print(f'*Set EF pi/2 pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma[qubit_i, qubit_i]}*')
results_Qi.update({'Pi/2 EF pulse gain':yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma[qubit_i, qubit_i]})
print(f'*New pi_ef.half_gain_pi_sigma\n', yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma)
yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma = yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma.flatten().astype(int).tolist()
print(f'*New pi_ef.half_gain_pi_sigma flattened\n{yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma}')
print(results_Qi)

In [ ]:
if save_data: saved_files.append(amprabi_4.save_data())

### EF $\pi/2$ error amplification (divide gain)

this is mostly only necessary for RB

In [80]:
divide_len = False

In [81]:
npulsecalib_efhalf = meas.NPulseExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"NPulseExptEF_half_Q{qubit_i}",
    config_file=config_path,
)

npulsecalib_efhalf.cfg = AttrDict(deepcopy(yaml_cfg))
npulsecalib_efhalf.cfg.expt = dict(
    start=0, # number gate repetitions
    step=1,
    expts=10,
    reps=1000,
    loops=20,
    pulse_type='gauss',
    test_pi_half=True,
    divide_len=divide_len,
    error_amp=True,

    checkEF=True, 
    qTest=qubit_i,
    qZZ=None,

    readout_ge=False, # whether to readout at the g/e set point (True) or e/f set point (False)

    post_process='scale',
    singleshot_reps=15000,
)

print(npulsecalib_efhalf.cfg)
import Pyro4.util
try:
    npulsecalib_efhalf.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

{'device': {'readout': {'trig_offset': 150, 'lo_sideband': [1, 1, 1, 1], 'frequency': [-389.975, 5795.761666663333, 502.368233333334, -228.9], 'gain': [0.4, 0.5, 0.3, 0.2], 'readout_length': [5, 20, 10, 8], 'frequency_ef': [-391.91408405, 5795.76166666, 501.35829955849954, -230.30732892], 'gain_ef': [0.05, 0.6, 0.4, 0.25], 'readout_length_ef': [1, 20, 10, 8], 'relax_delay': [450, 450, 35.0, 450]}, 'qubit': {'f_ge': [4108.6352143528575, 4103.243553024718, 4108.491689964749, 4108.066288400736, 3442.8688097468294, 3448.2597196742877, 3447.149851461427, 3446.943321983037, 4759.488589798401, 4758.50698515513, 4759.617265888647, 4759.085816492836, 4377.407821334912, 4376.654922763285, 4377.445747133665, 4377.972703369784], 'f_ef': [3882.584812861004, 0.0, 3882.593023159515, 3882.5465667921762, 3345.3957364303, 3347.8123422280078, 3347.003494996075, 3346.9817293387923, 4569.728581643396, 4568.311187621723, 4574.2134872558845, 0.0, 4206.708200302393, 4202.366145184353, 4205.110372632136, 4205.

  0%|          | 0/2 [00:00<?, ?it/s]

Qubit  (2)
g unrotated averages:
I -2.6927348307291665 +/- 16.821274500334624 	 Q 52.18564774305555 +/- 18.37743618499587 	 Amp 55.14232427615608 +/- 17.623908794684358
Rotated (theta=1.6266676203147412):
I -51.95384921458511 +/- 18.331499370534626 	 Q -5.60269604914053 +/- 16.87132381816234 	 Amp 52.25507297125228 +/- 17.623908794684358
e unrotated averages:
I -3.6022380208333336 +/- 17.808759295124734 	 Q 35.92404778645833 +/- 24.671910246245513 	 Amp 41.968203478441126 +/- 21.634192560507724
Rotated (theta=1.6266676203147412):
I -35.666835086722 +/- 24.4174789731615 	 Q -5.60269604914053 +/- 18.15603986076539 	 Amp 36.10420097609804 +/- 21.634192560507724
gf fidelity (%): 32.06666666666671 	 angle (deg): 93.20118931462373 	 threshold ge: -36.45607007030863
thresholds=[0, 0, -36.45607007030863, 0],
angles=[0, 0, 93.20118931462373, 0],
ge_avgs=[array([0., 0., 0., 0.]), array([0., 0., 0., 0.]), array([-2.69273483, 52.18564774, -3.60223802, 35.92404779]), array([0., 0., 0., 0.])],
count

  0%|          | 0/20 [00:00<?, ?it/s]

 s:\Seb\experiements\qram_tprocv1_expts\experiments\clifford_averager_program.py: 1877

In [82]:
# # TESTING
# temp_data, attrs = prev_data(expt_path, '00000_voltage_sweep_qubit_spectroscopy_qubit3.h5')
# data = lengthrabi.analyze(data=temp_data, fit=True)

data = npulsecalib_efhalf.data
npulsecalib_efhalf.analyze(fit=True, scale=None)
npulsecalib_efhalf.display(fit=True, scale=None)

# npulsecalib_efhalf_fit, npulsecalib_efhalf_fit_err = meas.fitting.get_best_fit(npulsecalib_efhalf.data)
fit_axis = 'avgi'
npulsecalib_efhalf_fit = data[f'fit_{fit_axis}']
npulsecalib_efhalf_fit_err = data[f'fit_err_{fit_axis}']

angle_err = npulsecalib_efhalf_fit[1]
amp_ratio = (90 - angle_err)/90
new_amp_half_ef = np.reshape(npulsecalib_efhalf.cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma, (4,4))[qubit_i, qubit_i] / amp_ratio

yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma, (4,4))
yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma[qubit_i, qubit_i] = int(round(new_amp_half_ef))
print(f'*Set pi/2 pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma[qubit_i, qubit_i]}*')
results_Qi.update({'Pi/2 pulse gain':yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma[qubit_i, qubit_i]})
print(f'*New pi_ef.half_gain_pi_sigma\n', yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma)
yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma = yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma.flatten().astype(int).tolist()
print(f'*New pi_ef.half_gain_pi_sigma flattened\n{yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma}')
print(results_Qi)

fitparams [None, 0.017453292519943295]
From amps: adjust amplitude to 3366 / 1.0254482615705323 = 3282.466923143231
From avgi: adjust amplitude to 3366 / 0.9963975372583366 = 3378.1697305894636
From avgq: adjust amplitude to 3366 / 1.0258321637515648 = 3281.2385095142863
*Set pi/2 pulse gain for qubit 2 to 3378*
*New pi_ef.half_gain_pi_sigma
 [[ 673    0    0    0]
 [   0  975    0    0]
 [   0    0 3378    0]
 [   0    0    0 1000]]
*New pi_ef.half_gain_pi_sigma flattened
[673, 0, 0, 0, 0, 975, 0, 0, 0, 0, 3378, 0, 0, 0, 0, 1000]
{'Qubit EF freq': 4574.2134872558845, 'T2R EF': 15.233515280933435, 'T2R EF err': 1.6586265411281977, 'Pi EF pulse gain': 13138, 'T2E EF': 21.632448119150048, 'T2E EF err': 1.6481093691006576, 'Pi/2 EF pulse gain': 30000, 'Pi/2 pulse gain': 3378, 'Res chi shift (e - g)': -0.9635761589403842, 'Res chi shift (f - g)': -1.009933774834451, 'Res freq ef (yaml)': 501.35829955849954, 'T1_EF': 8.82646968911557, 'T1_EF err': 2.176960259233138}


In [ ]:
if save_data: npulsecalib_efhalf.save_data()

## Resonator chi shift (e/f)

In [70]:
rspecEF_chi = meas.ResonatorSpectroscopyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"resonator_spectroscopy_chi_qubit{qubit_i}",
    config_file=config_path,
)
rspecEF_chi.cfg = AttrDict(deepcopy(yaml_cfg))

span = 3.5 # MHz
npts = 151
rspecEF_chi.cfg.expt = dict(
    start=rspecEF_chi.cfg.device.readout.frequency[qubit_i]-abs(results_Qi['Res chi shift (e - g)'])-span, # [MHz]
    # start=rspecEF_chi.cfg.device.readout.frequency[qubit_i]-span/2, # [MHz]
    step=span/npts,
    expts=npts,
    reps=reps*rounds,
    pulse_e=True, # add ge pi pulse prior to measurement
    pulse_f=True, # add ef pi pulse prior to measurement
    qTest=qubit_i,
    qZZ=None,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
)

# rspecEF_chi.cfg.device.readout.gain[qubit_i] /= 2
rspecEF_chi.cfg.device.readout.gain = rspecEF_chi.cfg.device.readout.gain_ef
# rspecEF_chi.cfg.device.readout.relax_delay = 100 # Wait time between experiments [us]
rspecEF_chi.go(analyze=False, display=False, progress=True, save=False)

  0%|          | 0/151 [00:00<?, ?it/s]

In [71]:
# # TESTING
# # rspecEF_chi = meas.ResonatorSpectroscopyExperiment(config_file=config_path,) 
# temp_data, attrs = prev_data(expt_path, '00001_resonator_spectroscopy_qubit2.h5')
# rspecEF_chi.data = temp_data

rspecEF_chi.data = rspecEF_chi.analyze(fit=True)
rspecEF_chi_fit = rspecEF_chi.data['fit']
rspecEF_chi.display(fit=True)
# res_freq_f = rspecEF_chi_fit[0] - yaml_cfg.hw.soc.dacs.readout.mixer_freq[qubit_i]
res_freq_f = rspecEF_chi.data['xpts'][np.argmin(rspecEF_chi.data['amps'])]
print('ef chi shift', yaml_cfg.device.readout.lo_sideband[qubit_i]*(res_freq_f - yaml_cfg.device.readout.frequency[qubit_i]))
results_Qi.update({'Res chi shift (f - g)': yaml_cfg.device.readout.lo_sideband[qubit_i]*(res_freq_f - yaml_cfg.device.readout.frequency[qubit_i])})
results_Qi.update({'Res freq ef (yaml)':res_freq_f})

print('Res freq ef:', res_freq_f)

# yaml_cfg.device.readout.frequency_ef = [0]*4
yaml_cfg.device.readout.frequency_ef[qubit_i] = res_freq_f
# res_freq_f = 6953.5
print(all_results)

electrical delay phase correction (no mux support) (deg) 1.013997774929151
param guess [7699.643067770418, 38498.21533885209, 38498.21533885209, 0, -0.09045325747865562, 0.9068244341556702]
param bounds ([7697.92783598234, 0, 0, -6.283185307179586, -0.13567988621798344, -inf], [7701.358299558499, 1000000000.0, 100000000.0, 6.283185307179586, 0.13567988621798344, 1.0484732150041731])
popt [ 7.69912235e+03  1.21206470e+05  3.15297130e+07  4.62609181e+00
 -4.71148418e-02 -6.86734904e-02]

Freq with minimum transmission: 501.33512075055256
Freq with maximum transmission: 497.9973724061817
From fit:
	f0: 7699.122347781455
	Qi: 121206.47007084842 	 kappa_i/2pi: 0.06352072082687593
	Qe: 31529713.047705833 	 kappa_e/2pi: 0.00024418624857550547
	Q0: 120742.31267476443
	kappa [MHz]: 0.06376490707545142
	phi [radians]: 4.626091811209294
baseline 82.54489619799361
ef chi shift -1.009933774834451
Res freq ef: 501.35829955849954
[{}, {}, {'Qubit EF freq': 4574.2134872558845, 'T2R EF': 15.23351528093

In [ ]:
if save_data: saved_files.append(rspecEF_chi.save_data())

## $T_1$ EF

In [78]:
t1EF = meas.T1Experiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"t1EF_qubit{qubit_i}",
    config_file=config_path,
)
t1EF.cfg = AttrDict(deepcopy(yaml_cfg))

span = 100
npts = 121

t1EF.cfg.expt = dict(
    start=0, # wait time [us]
    step=span/npts,
    expts=npts,
    reps=reps*rounds*4,
    rounds=1,
    qTest=qubit_i,
    qZZ=None,
    # qZZ=0,
    checkEF=True,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
)

# for q in range(4):
#     if q != qubit_i:
#         t1EF.cfg.device.readout.gain[q] = 1e-4

t1EF.cfg.device.readout.relax_delay = span # Wait time between experiments [us]
t1EF.go(analyze=False, display=False, progress=True, save=False)

  0%|          | 0/484000 [00:00<?, ?it/s]

In [79]:
# # TESTING
# # t1EF = meas.T1Experiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_t1EF_qubit1.h5')
# t1EF.data = temp_data

t1EF.analyze(fit_log=False)

t1EF_fit, t1EF_fit_err = meas.fitting.get_best_fit(t1EF.data) #, fitter.expfunc)

t1EF.display(fit=True, fit_log=False)
t1EF_qubit = t1EF_fit[3]
yaml_cfg.device.readout.relax_delay[qubit_i] = np.round(4*t1EF_qubit)
print(f'*Set readout {qubit_i} relax delay to {yaml_cfg.device.readout.relax_delay[qubit_i]}*')
results_Qi.update({'T1_EF':t1EF_qubit, 'T1_EF err':np.sqrt(t1EF_fit_err[3][3])})
print(f'qubit {qubit_i}', results_Qi)

best 1 1502433.3910973358
Fit T1 amps [us]: 19.834710743801654
Fit T1 avgi [us]: 8.82646968911557
Fit T1 avgq [us]: 19.834710743801654
*Set readout 2 relax delay to 35.0*
qubit 2 {'Qubit EF freq': 4574.2134872558845, 'T2R EF': 15.233515280933435, 'T2R EF err': 1.6586265411281977, 'Pi EF pulse gain': 13138, 'T2E EF': 21.632448119150048, 'T2E EF err': 1.6481093691006576, 'Pi/2 EF pulse gain': 30000, 'Pi/2 pulse gain': 3366, 'Res chi shift (e - g)': -0.9635761589403842, 'Res chi shift (f - g)': -1.009933774834451, 'Res freq ef (yaml)': 501.35829955849954, 'T1_EF': 8.82646968911557, 'T1_EF err': 2.176960259233138}


In [ ]:
if save_data: saved_files.append(t1.save_data())

# Single Shot

In [18]:
import yaml
with open(config_path, 'r') as cfg_file:
    yaml_cfg = yaml.safe_load(cfg_file)
yaml_cfg = AttrDict(yaml_cfg)

## Single shot optimization

In [ ]:
results_Qi['Res kappa'] = 0.2
# print(results_Qi['Res kappa'])

In [131]:
qubit_i = 3

In [133]:
shotopt = meas.SingleShotOptExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"single_shot_opt_qubit{qubit_i}",
    config_file=config_path,
)
shotopt.cfg = AttrDict(deepcopy(yaml_cfg))

# span_f = 2*abs(results_Qi['Res chi shift (f - g)']) # MHz
# span_f = 2*abs(results_Qi['Res chi shift (e - g)']) # MHz

span_f = 1.0*abs(results_Qi['Res kappa']) # MHz
npts_f = 15
# span_f = 0
# npts_f = 1

span_gain = 0.7
npts_gain = 8
# span_gain = 0.0
# npts_gain = 8

# span_len = 5 # us
# npts_len = 10
span_len = 14 # us
npts_len = 4

shotopt.cfg.expt = dict(
    reps=1000,
    qTest=qubit_i,

    start_f=shotopt.cfg.device.readout.frequency[qubit_i] - span_f/2, # [MHz]
    step_f=span_f/npts_f,
    expts_f=npts_f,

    # start_gain=shotopt.cfg.device.readout.gain[qubit_i]-span_gain/2,
    start_gain=0.05,
    step_gain=span_gain/npts_gain,
    expts_gain=npts_gain + 1,

    start_len=1,
    # start_len=shotopt.cfg.device.readout.readout_length[qubit_i] - span_len/2,
    step_len=span_len/npts_len,
    expts_len=npts_len,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
)

print(shotopt.cfg)

shotopt.go(analyze=False, display=False, progress=True, save=False)

{'device': {'readout': {'trig_offset': 150, 'lo_sideband': [1, 1, 1, 1], 'frequency': [-390.00045219204236, 5795.770150727347, 502.42095511476606, -228.9], 'gain': [0.8, 0.79, 0.5, 0.2], 'readout_length': [4.5, 16, 8.5, 8], 'frequency_ef': [-391.91408405, 5795.76166666, 500.44439227, -230.30732892], 'gain_ef': [0.05, 0.6, 0.4, 0.25], 'readout_length_ef': [1, 20, 10, 8], 'relax_delay': [450, 450, 450, 450]}, 'qubit': {'f_ge': [4108.6352143528575, 4103.243553024718, 4108.491689964749, 4108.066288400736, 3442.8688097468294, 3448.2597196742877, 3447.149851461427, 3446.943321983037, 4759.488589798401, 4758.50698515513, 4759.617265888647, 4759.085816492836, 4377.407821334912, 4376.654922763285, 4377.445747133665, 4377.972703369784], 'f_ef': [3882.584812861004, 0.0, 3882.593023159515, 3882.5465667921762, 3345.3957364303, 3347.8123422280078, 3347.003494996075, 3346.9817293387923, 4569.728581643396, 4568.311187621723, 4570.125075914433, 0.0, 4206.708200302393, 4202.366145184353, 4205.1103726321

  0%|          | 0/15 [00:00<?, ?it/s]

freq: -229.02726096021163, gain: 0.05, len: 1.0
	fid [%]: 2.300000000000002
freq: -229.02726096021163, gain: 0.05, len: 4.5
	fid [%]: 11.8396396396397
freq: -229.02726096021163, gain: 0.05, len: 8.0
	fid [%]: 10.800000000000004
freq: -229.02726096021163, gain: 0.05, len: 11.5
	fid [%]: 17.600000000000016
freq: -229.02726096021163, gain: 0.1375, len: 1.0
	fid [%]: 9.200000000000008
freq: -229.02726096021163, gain: 0.1375, len: 4.5
	fid [%]: 31.600000000000023
freq: -229.02726096021163, gain: 0.1375, len: 8.0
	fid [%]: 33.10000000000003
freq: -229.02726096021163, gain: 0.1375, len: 11.5
	fid [%]: 37.90000000000002
freq: -229.02726096021163, gain: 0.22499999999999998, len: 1.0
	fid [%]: 12.742942942943003
freq: -229.02726096021163, gain: 0.22499999999999998, len: 4.5
	fid [%]: 45.30000000000003
freq: -229.02726096021163, gain: 0.22499999999999998, len: 8.0
	fid [%]: 53.00000000000005
freq: -229.02726096021163, gain: 0.22499999999999998, len: 11.5
	fid [%]: 54.70000000000004
freq: -229.027

In [134]:
imax = shotopt.analyze()
shotopt.display()
print(shotopt.data['fid'])

freq_opt = shotopt.data['fpts'][imax[0]]
gain_opt = shotopt.data['gainpts'][imax[1]]
len_opt = shotopt.data['lenpts'][imax[2]]

print('Optimal freq:', freq_opt)
print('Optimal gain:', gain_opt)
print('Optimal len:', len_opt)

(5, 7, 1)
[-229.02726096 -229.01029283 -228.9933247  -228.97635658 -228.95938845
 -228.94242032 -228.92545219 -228.90848406 -228.89151594 -228.87454781
 -228.85757968 -228.84061155 -228.82364342 -228.8066753  -228.78970717]
[0.05   0.1375 0.225  0.3125 0.4    0.4875 0.575  0.6625 0.75  ]
[ 1.   4.5  8.  11.5]
Max fidelity 86.90000000000005 %
Set params: 
 angle (deg) -169.08761857077386 
 threshold 130.2514356508114 
 freq [Mhz] -228.94242032007057 
 gain [dac units] 0.6625 
 readout length [us] 4.5
[[[0.023      0.1183964  0.108      0.176     ]
  [0.092      0.316      0.331      0.379     ]
  [0.12742943 0.453      0.53       0.547     ]
  [0.163      0.572      0.62       0.62862863]
  [0.181      0.662      0.71083884 0.72477077]
  [0.19276076 0.733      0.733      0.725     ]
  [0.202      0.79       0.80295996 0.72      ]
  [0.24454154 0.82696597 0.791      0.75494494]
  [0.274      0.843      0.825      0.737     ]]

 [[0.02687187 0.089      0.144      0.164     ]
  [0.097     

In [92]:

# Using readlines()
file1 = open('C:\\Users\\slab\\Downloads\\singleshot_opt.txt', 'r')
lines = file1.readlines()
freqs = []
gains = []
lengths = []
fids = []

line = lines[0]
words = line.split()
freq = float(words[1][:-1])
freqs.append(freq)
gain = float(words[3][:-1])
gains.append(gain)
length = float(words[5])
lengths.append(length)

fid_line = lines[1]
fids.append(float(fid_line.split()[3]))
 
# Strips the newline character
for i in range(2, len(lines), 2):
    line = lines[i]
    words = line.split()
    freq = float(words[1][:-1])
    if freq not in freqs: freqs.append(freq)
    gain = float(words[3][:-1])
    if gain not in gains: gains.append(gain)
    length = float(words[5])
    if length not in lengths: lengths.append(length)

    fid_line = lines[i+1]
    fids.append(float(fid_line.split()[3]))
    continue

fids = np.append(fids, [0]*22)
fids = np.reshape(fids, newshape=(len(freqs), len(gains), len(lengths)))

# lenpts = [data['lenpts'][0]]
for g_ind, gain in enumerate(gains):
    for l_ind, l in enumerate(lengths):
        plt.plot(freqs, fids[:,g_ind, l_ind], 'o-', label=f'gain: {gain:.2}, len [us]: {l}')
plt.xlabel('Frequency [MHz]')
plt.ylabel(f'Fidelity [%]')
plt.legend()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\slab\\Downloads\\singleshot_opt.txt'

In [ ]:
if save_data: saved_files.append(shotopt.save_data())

next qubit

In [ ]:
qubit_i = 2
results_Qi = all_results[qubit_i]
results_Qi['Res kappa'] = 0.2
# print(results_Qi['Res kappa'])

In [ ]:
shotopt = meas.SingleShotOptExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"single_shot_opt_qubit{qubit_i}",
    config_file=config_path,
)
shotopt.cfg = AttrDict(deepcopy(yaml_cfg))

# span_f = 2*abs(results_Qi['Res chi shift (f - g)']) # MHz
# span_f = 2*abs(results_Qi['Res chi shift (e - g)']) # MHz

# span_f = 2*abs(results_Qi['Res kappa']) # MHz
# npts_f = 15

span_f = 3*abs(results_Qi['Res kappa']) # MHz
npts_f = 20

# span_gain = 0.85
# npts_gain = 5
span_gain = 0.6
npts_gain = 6

span_len = 0 # us
npts_len = 1

shotopt.cfg.expt = dict(
    reps=20000,
    qTest=qubit_i,

    # start_f=shotopt.cfg.device.readout.frequency[qubit_i] - 0.75*span_f,
    # start_f=shotopt.cfg.device.readout.frequency[qubit_i] + 0.25*span_f,
    # start_f=shotopt.cfg.device.readout.frequency[qubit_i],
    start_f=shotopt.cfg.device.readout.frequency[qubit_i] - span_f/2, # [MHz]
    step_f=span_f/npts_f,
    expts_f=npts_f,

    # start_gain=int(shotopt.cfg.device.readout.gain[qubit_i]-span_gain/2),
    start_gain=0.1,
    # start_gain=0.10,
    step_gain=span_gain/npts_gain,
    expts_gain=npts_gain + 1,

    start_len=5,
    # step_len=span_len/npts_len,
    # expts_len=npts_len + 1,
    step_len=0,
    expts_len=1,
)

print(shotopt.cfg)

shotopt.go(analyze=False, display=False, progress=True, save=False)

In [124]:
imax = shotopt.analyze()
shotopt.display()
print(shotopt.data['fid'])

freq_opt = shotopt.data['fpts'][imax[0]]
gain_opt = shotopt.data['gainpts'][imax[1]]
len_opt = shotopt.data['lenpts'][imax[2]]

print(freq_opt, gain_opt, len_opt, shotopt.data['fid'][imax])

(8, 8, 2)
[5795.6344057  5795.65137383 5795.66834196 5795.68531009 5795.70227822
 5795.71924634 5795.73621447 5795.7531826  5795.77015073 5795.78711886
 5795.80408698 5795.82105511 5795.83802324 5795.85499137 5795.8719595 ]
[0.1    0.1875 0.275  0.3625 0.45   0.5375 0.625  0.7125 0.8   ]
[ 1.   8.5 16.  23.5]
Max fidelity 79.19299299299294 %
Set params: 
 angle (deg) -8.065614956885108 
 threshold 79.399318933365 
 freq [Mhz] 5795.770150727347 
 gain [dac units] 0.7999999999999999 
 readout length [us] 16.0
[[[0.048      0.1        0.13453153 0.118     ]
  [0.044      0.118      0.197      0.243     ]
  [0.067      0.158      0.285      0.20675375]
  [0.058      0.25372773 0.287      0.38853307]
  [0.044      0.323      0.378      0.439     ]
  [0.093      0.345      0.50073173 0.443     ]
  [0.108      0.51870771 0.484      0.401     ]
  [0.128      0.41       0.576      0.498     ]
  [0.143      0.503      0.61865531 0.638     ]]

 [[0.041      0.099      0.071      0.098     ]
  [0.

In [ ]:
if save_data: saved_files.append(shotopt.save_data())

next qubit

In [ ]:
qubit_i = 3
results_Qi = all_results[qubit_i]
results_Qi['Res kappa'] = 0.2
# print(results_Qi['Res kappa'])

In [85]:
shotopt = meas.SingleShotOptExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"single_shot_opt_qubit{qubit_i}",
    config_file=config_path,
)
shotopt.cfg = AttrDict(deepcopy(yaml_cfg))

# span_f = 2*abs(results_Qi['Res chi shift (f - g)']) # MHz
# span_f = 2*abs(results_Qi['Res chi shift (e - g)']) # MHz

# span_f = 2*abs(results_Qi['Res kappa']) # MHz
# npts_f = 15

span_f = 3*abs(results_Qi['Res kappa']) # MHz
npts_f = 20

# span_gain = 0.85
# npts_gain = 5
span_gain = 0.1
npts_gain = 8

span_len = 0 # us
npts_len = 1

shotopt.cfg.expt = dict(
    reps=15000,
    qTest=qubit_i,

    # start_f=shotopt.cfg.device.readout.frequency[qubit_i] - 0.75*span_f,
    # start_f=shotopt.cfg.device.readout.frequency[qubit_i] + 0.25*span_f,
    # start_f=shotopt.cfg.device.readout.frequency[qubit_i],
    start_f=shotopt.cfg.device.readout.frequency[qubit_i] - span_f/2, # [MHz]
    step_f=span_f/npts_f,
    expts_f=npts_f,

    # start_gain=int(shotopt.cfg.device.readout.gain[qubit_i]-span_gain/2),
    start_gain=0.01,
    # start_gain=0.10,
    step_gain=span_gain/npts_gain,
    expts_gain=npts_gain + 1,

    start_len=8.0,
    # step_len=span_len/npts_len,
    # expts_len=npts_len + 1,
    step_len=0,
    expts_len=1,
)

print(shotopt.cfg)

shotopt.go(analyze=False, display=False, progress=True, save=False)

KeyError: 'Res kappa'

In [ ]:
imax = shotopt.analyze()
shotopt.display()
print(shotopt.data['fid'])

freq_opt = shotopt.data['fpts'][imax[0]]
gain_opt = shotopt.data['gainpts'][imax[1]]
len_opt = shotopt.data['lenpts'][imax[2]]

In [ ]:
if save_data: saved_files.append(shotopt.save_data())

## Best readout parameters

In [18]:
qubit_i = 3

In [ ]:
shot = meas.HistogramExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"single_shot_qubit{qubit_i}",
    config_file=config_path,
)
shot.cfg = AttrDict(deepcopy(yaml_cfg))

shot.cfg.expt = dict(
    reps=8000,
    check_f=False,
    qTest=qubit_i,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
)

# shot.cfg.device.readout.frequency=freq_opt
shot.cfg.device.readout.gain[0]=0.6
shot.cfg.device.readout.gain[1]=0.6
shot.cfg.device.readout.gain[2]=0.5
shot.cfg.device.readout.gain[3]=0.3
# print(freq_opt, gain_opt)
# shot.cfg.device.readout.readout_length = len_opt # [us]

# shot.cfg.device.readout.frequency=-807.4996158081035
# shot.cfg.device.readout.readout_length = 10 # [us]
# shot.cfg.device.readout.relax_delay = 70 # Wait time between experiments [us]
print(shot.cfg)

shot.go(analyze=False, display=False, progress=True, save=False)

{'device': {'readout': {'trig_offset': 150, 'lo_sideband': [1, 1, 1, 1], 'frequency': [-390.00045219204236, 5795.770150727347, 502.42095511476606, -228.94242032007057], 'gain': [0.6, 0.6, 0.5, 0.3], 'readout_length': 10, 'frequency_ef': [-391.91408405, 5795.76166666, 500.44439227, -230.30732892], 'gain_ef': [0.05, 0.6, 0.4, 0.25], 'readout_length_ef': [1, 20, 10, 8], 'relax_delay': [450, 450, 450, 450]}, 'qubit': {'f_ge': [4108.6352143528575, 4103.243553024718, 4108.491689964749, 4108.066288400736, 3442.8688097468294, 3448.2597196742877, 3447.149851461427, 3446.943321983037, 4759.488589798401, 4758.50698515513, 4759.617265888647, 4759.085816492836, 4377.407821334912, 4376.654922763285, 4377.445747133665, 4377.972703369784], 'f_ef': [3882.584812861004, 0.0, 3882.593023159515, 3882.5465667921762, 3345.3957364303, 3347.8123422280078, 3347.003494996075, 3346.9817293387923, 4569.728581643396, 4568.311187621723, 4570.125075914433, 0.0, 4206.708200302393, 4202.366145184353, 4205.110372632136,

  0%|          | 0/8000 [00:00<?, ?it/s]

  0%|          | 0/8000 [00:00<?, ?it/s]

In [30]:
# # TESTING
# shot = meas.HistogramExperiment(config_file=config_path)
# temp_data, attrs = prev_data(expt_path, '00001_single_shot_qubit0.h5')
# data = shot.analyze(data=temp_data)
# shot.display(data=data)

shot.analyze(verbose=False)
shot.display(verbose=True, amplitude_mode=shot.cfg.expt.full_mux_expt)
results_Qi.update({'Single shot g-e fidelity (%)': 100*shot.data['fids'][0]})

g unrotated averages:
I -96.65661690266928 +/- 23.89468269238032 	 Q 64.96045206705729 +/- 24.27172830673666 	 Amp 119.21267221309536 +/- 22.600295653032536
Rotated (theta=-0.8165673748901313):
I -18.840229715532992 +/- 25.501966291186548 	 Q 114.92348614661742 +/- 22.577031940641938 	 Amp 116.45755417243579 +/- 22.600295653032536
e unrotated averages:
I -45.69408414713542 +/- 27.180604870504514 	 Q 119.20321321614584 +/- 31.590814396985138 	 Amp 131.76086052442025 +/- 25.946017735963732
Rotated (theta=-0.8165673748901313):
I 55.58729800060125 +/- 33.76360971955852 	 Q 114.92348614661744 +/- 24.429152545608098 	 Amp 127.66109574611765 +/- 25.946017735963732
ge fidelity (%): 81.5993155394424
rotation angle (deg): -46.7858642692814
threshold ge: 18.924971339815087


In [ ]:
if save_data: saved_files.append(shot.save_data())

In [ ]:
# TESTING
shot = meas.HistogramExperiment(config_file=config_path)
temp_data, attrs = prev_data('S:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00000_single_shot_qubit2.h5')
shot.cfg = AttrDict(attrs['config'])
shot.data = temp_data

In [ ]:
# Checking for drift vs. shot number

data = shot.data

Ies = data['Ie']
Qes = data['Qe']
Igs = data['Ig']
Qgs = data['Qg']

plot_e = False

if plot_e:
    i, q = Ies, Qes
else:
    i, q = Igs, Qgs

amps = np.abs(i+1j*q)
phases = np.angle(i+1j*q)

# plt.plot(amps, '.', label='amps')
# plt.ylabel('Amps [ADC units]')
# plt.xlabel('Shot number')
# plt.legend()
# plt.show()

plt.title(f'Q{shot.cfg.expt.qTest} {"e" if plot_e else "g"}')
plt.plot(phases, '.', label='phases')
plt.axhline(0, color='r')
# plt.axhline(1.5, color='r')
plt.ylabel('Phases [radians]')
plt.xlabel('Shot number')
plt.legend()
plt.show()

plt.plot(i, '.', label='i')
plt.ylabel('I [ADC units]')
plt.xlabel('Shot number')
plt.legend()
plt.show()

plt.plot(q, '.', label='Q')
plt.ylabel('Q [ADC units]')
plt.xlabel('Shot number')
plt.legend()
plt.show()

# Results

In [177]:
print(yaml_cfg)
for Qi, results in enumerate(all_results):
    print(f'Qubit {Qi}')
    for key, value in results.items():
        print(f'\t{key}: {value}')
    print()

for saved_file in saved_files:
    print(saved_file)

print(f'f_ge\n{yaml_cfg.device.qubit.f_ge}')
print(f'pi_ge.gain\n{yaml_cfg.device.qubit.pulses.pi_ge.gain}')
print(f'pi_ge.half_gain\n{yaml_cfg.device.qubit.pulses.pi_ge.half_gain}')
print(f'pi_ge.half_gain_pi_sigma\n{yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma}')
print(f'f_ef\n{yaml_cfg.device.qubit.f_ef}')
print(f'pi_ef.gain\n{yaml_cfg.device.qubit.pulses.pi_ef.gain}')
print(f'pi_ef.half_gain\n{yaml_cfg.device.qubit.pulses.pi_ef.half_gain}')
print(f'pi_ef.half_gain_pi_sigma\n{yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma}')
print(f'readout.frequency_ef\n{yaml_cfg.device.readout.frequency_ef}')

{'device': {'readout': {'trig_offset': 150, 'lo_sideband': [1, 1, 1, 1], 'frequency': [-389.9616666666667, 5795.78833333, 502.36156666666733, -228.84666666666666], 'gain': [0.4, 0.5, 0.3, 0.2], 'readout_length': [5, 20, 10, 8], 'frequency_ef': [-391.91408405, 5795.76166666, 500.44439227, -230.30732892], 'gain_ef': [0.05, 0.6, 0.4, 0.25], 'readout_length_ef': [1, 20, 10, 8], 'relax_delay': [450, 259.0, 171.0, 124.0]}, 'qubit': {'f_ge': [4111.107346135244, 4105.758870288914, 4110.961390890517, 4110.561006068113, 3442.9791477220547, 3448.258871857906, 3447.2256002913127, 3447.022898974775, 4762.736987583428, 4761.772206046936, 4759.604749454737, 4762.340227746316, 4380.990256112639, 4380.2241341449835, 4380.999038792971, 4377.984990507439], 'f_ef': [3884.9668600899413, 0.0, 0.0, 0.0, 0.0, 3347.846225737386, 0.0, 0.0, 4571.9056482340775, 4571.009319367402, 4570.352063457543, 0.0, 4211.704645679207, 4206.869581627414, 0.0, 4204.964704068256], 'f_EgGf': [2686.53594683, 0, 2017.3, 2414.928509